<a href="https://colab.research.google.com/github/1235357/PyFigureEditor/blob/main/LEARNING_Project_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 PyFigureEditor 教学版 - 完整代码逐行解析

### 🎯 项目核心问题解决

**传统 Python 绑图的痛点：**
```python
# 每次修改都需要改代码、重新运行
plt.plot([1,2,3], [4,5,6], color='blue')  # 想改成红色？
# 1. 回到代码 → 2. 改成 color='red' → 3. 重新运行 → 4. 不满意？重复以上... 😫
```

**我的解决方案 - PyFigureEditor：**
```
无需修改代码！点击图表 → 选择元素 → 修改属性 → 即时生效！
像在 PowerPoint 里编辑图片一样简单 ✨
```

---

### 🔑 Presentation 关键技术点预览

1. **Dash 响应式框架** - 类似 React 的 Python Web 框架
2. **Callback 机制** - 用户操作自动触发函数执行
3. **状态管理模式** - 类似 Redux 的集中式状态管理
4. **Plotly 图表对象** - 可编程的交互式图表

---

### ❓ 预期的 Q&A 问题分类

| 问题类别 | 可能的问题示例 | 对应Part |
|----------|----------------|----------|
| **为什么选择这个技术栈？** | "为什么用 Dash 而不是 Flask？" | Part 1 |
| **架构设计** | "FigureStore 的设计灵感来自哪里？" | Part 2 |
| **技术难点** | "Callback 冲突怎么解决？" | Part 5 |
| **扩展性** | "如何添加新的图表类型？" | Part 3 |
| **实际应用** | "这个工具的目标用户是谁？" | 总结 |

---

⬇️ **现在开始逐步学习代码！**

---
# 📦 Part 1: 基础设置与核心概念
## 1.1 库导入与环境初始化

**学习目标：**
- 理解项目使用的核心库及其用途
- 学会初始化 Dash 应用
- 理解为什么需要这些配置

---

### 📚 核心库对照表

| 库名 | 用途 | 为什么选它？ | 类似工具对比 |
|------|------|-------------|--------------|
| `dash` | Web应用框架 | Python原生，无需学JS | Flask需要写前端，Streamlit功能弱 |
| `plotly` | 交互式图表 | 支持26+种图表，可导出 | Matplotlib不交互，Bokeh复杂 |
| `pandas` | 数据处理 | 业界标准，功能全面 | 无替代品 |
| `numpy` | 数值计算 | 快速，支持向量化 | 无替代品 |
| `dash_bootstrap_components` | UI美化 | 开箱即用的美观组件 | 原生Dash样式丑 |

---

### ❓ 可能的提问与回答

**Q1: 为什么选择 Dash 而不是 Flask 或 Django？**
> **A:** Flask/Django 是通用Web框架，需要自己处理前端渲染和交互。Dash 专为数据可视化设计，内置了响应式更新机制（Callback），不需要写 JavaScript，一个Python文件就能完成完整应用。

**Q2: 为什么用 Plotly 而不是 Matplotlib？**
> **A:** Matplotlib 是静态图表库，每次修改需要重新生成图片。Plotly 生成的是交互式 HTML/JS 图表，支持缩放、悬停提示、导出等功能，且能无缝嵌入 Dash。

**Q3: `suppress_callback_exceptions=True` 是什么意思？**
> **A:** 这允许我们定义指向"还不存在的组件"的 Callback。因为属性面板是动态生成的，如果没有这个设置，Dash 启动时会报错"找不到组件"。

**Q4: 为什么使用中国CDN？**
> **A:** 默认的 Bootstrap CDN 在中国可能被墙或速度慢，换成 `cdn.staticfile.org` 确保样式能正常加载。

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  Part 1.1: 库导入与环境初始化                                                  ║
# ║  这是整个应用的基础设施层                                                       ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ============================================================================
# 🔹 核心框架导入 - Dash 生态系统
# ============================================================================
#
# 【关键概念】Dash 是什么？
# Dash = Flask (后端) + React.js (前端) + Plotly (图表)
# 用纯 Python 就能写出交互式 Web 应用！
#
# 【类比理解】
# 如果做一道菜：
# - Flask = 锅（基础容器）
# - React = 翻炒技术（用户交互）
# - Plotly = 食材（数据可视化）
# - Dash = 整个烹饪系统（全自动化）

import dash                    # Dash 框架主模块 - 构建Web应用的核心
                               # 版本要求: >= 2.17.0

from dash import (
    html,                      # HTML 组件 - 对应原生 HTML 标签
                               # 例: html.Div() → <div></div>
                               # 例: html.H1("标题") → <h1>标题</h1>

    dcc,                       # Dash Core Components - 高级交互组件
                               # 例: dcc.Dropdown() - 下拉菜单
                               # 例: dcc.Graph() - Plotly 图表容器
                               # 例: dcc.Slider() - 滑动条

    Input,                     # Callback 输入装饰器
                               # 当这个组件的值改变时，触发回调函数
                               # 例: Input('button', 'n_clicks') - 按钮被点击

    Output,                    # Callback 输出装饰器
                               # 回调函数的返回值会更新这个组件
                               # 例: Output('graph', 'figure') - 更新图表

    State,                     # Callback 状态装饰器
                               # 读取组件的当前值，但不触发回调
                               # 例: State('input', 'value') - 读取输入框内容

    ctx,                       # Callback 上下文
                               # 用于判断是哪个 Input 触发了回调
                               # 例: ctx.triggered_id == 'btn-save'

    dash_table,                # 数据表格组件 - 展示 DataFrame
                               # 支持排序、筛选、编辑

    ALL,                       # 模式匹配 - 匹配所有同类组件
                               # 用于动态生成的多个相似组件

    MATCH                      # 模式匹配 - 匹配特定ID的组件
                               # 用于识别具体是哪个动态组件
)

import dash_bootstrap_components as dbc
# Bootstrap 组件库 - 提供美观的 UI 组件
# 例: dbc.Button() - 漂亮的按钮
# 例: dbc.Card() - 卡片容器
# 例: dbc.Modal() - 弹窗对话框
# 【重要】没有这个库，Dash 界面会非常简陋！

# ============================================================================
# 🔹 数据可视化库 - Plotly 生态系统
# ============================================================================
#
# 【关键概念】Plotly 的两种使用方式：
# 1. plotly.express (px) - 高级接口，一行代码出图，适合快速探索
# 2. plotly.graph_objects (go) - 底层接口，精细控制每个元素
#
# 【类比理解】
# px = 自动挡汽车（简单但控制少）
# go = 手动挡汽车（复杂但控制多）

import plotly                  # Plotly 主模块 - 用于获取版本号等
import plotly.graph_objects as go
# 底层图表对象 - 精细控制每个元素
# go.Figure() - 创建空图表
# go.Scatter() - 散点/折线图
# go.Bar() - 柱状图
# go.Pie() - 饼图
# 【重要】我们的编辑器主要使用 go，因为需要精细控制！

import plotly.express as px
# 高级接口 - 快速创建图表
# px.scatter(df, x='col1', y='col2') - 一行代码创建散点图
# 【用途】自动生成绘图代码时使用

# ============================================================================
# 🔹 数据处理库 - 数据科学核心
# ============================================================================

import pandas as pd
# Pandas - 表格数据处理的王者
# pd.DataFrame - 二维表格数据结构
# pd.read_csv() - 读取CSV文件
# 【重要】所有用户数据都存储为 DataFrame！

import numpy as np
# NumPy - 数值计算的基础
# np.array - 高效数组
# np.linspace() - 生成等间距数列
# np.sin(), np.cos() - 数学函数
# 【用途】生成演示数据、数值计算

# ============================================================================
# 🔹 Python 标准库 - 内置工具箱
# ============================================================================

import json
# JSON 序列化/反序列化
# json.dumps() - Python对象 → JSON字符串
# json.loads() - JSON字符串 → Python对象
# 【用途】保存/加载会话、与前端通信

import base64
# Base64 编码/解码
# base64.b64encode() - 二进制 → 文本
# base64.b64decode() - 文本 → 二进制
# 【用途】处理用户上传的 CSV 文件

import io
# 内存中的文件操作
# io.StringIO() - 字符串当文件读
# io.BytesIO() - 字节流当文件读
# 【用途】解析上传的文件内容

from dataclasses import dataclass, field
# Python 3.7+ 数据类装饰器
# @dataclass 自动生成 __init__, __repr__, __eq__
# 【好处】减少样板代码，代码更简洁

from datetime import datetime
# 日期时间处理
# datetime.now() - 当前时间
# 【用途】记录操作时间、元数据

from collections import deque, defaultdict
# 高效集合类型
# deque - 双端队列，用于实现撤销栈
# defaultdict - 带默认值的字典
# 【用途】历史记录管理

from typing import Dict, List, Any, Optional, cast, Sequence
# 类型提示 - 让代码更可读、IDE更智能
# Dict[str, Any] - 字符串键的字典
# List[int] - 整数列表
# Optional[str] - 可能为 None 的字符串
# 【好处】帮助发现bug，提高代码质量

import copy
# 对象拷贝
# copy.deepcopy() - 完全独立的副本
# 【用途】保存历史状态时，必须深拷贝！

import uuid
# 唯一标识符生成
# uuid.uuid4() - 随机生成 UUID
# 【用途】避免应用名称冲突

from dash.exceptions import PreventUpdate
# 特殊异常 - 阻止回调更新
# raise PreventUpdate - 告诉 Dash "这次不需要更新"
# 【用途】条件性跳过回调，避免不必要的更新

import contextlib
# 上下文管理工具
# @contextlib.suppress(Exception) - 忽略特定异常
# 【用途】优雅地处理错误

# ============================================================================
# 🔹 设置随机种子 - 确保每次运行生成相同的演示数据
# ============================================================================
#
# 【问题】np.random 生成的是"伪随机数"，每次运行结果不同
# 【解决】设置种子后，随机数序列固定，便于调试和演示
# 【为什么是42？】来自《银河系漫游指南》的梗，"42是生命、宇宙和一切的答案"

np.random.seed(42)

# ============================================================================
# 🔹 初始化 Dash 应用 - 核心配置
# ============================================================================

# 生成唯一的应用名称
# 【问题】Jupyter 重复运行时，同名应用会冲突
# 【解决】每次生成随机后缀，确保唯一性
app_name = f"Interactive_Editor_{uuid.uuid4().hex[:8]}"

# 使用国内CDN加速Bootstrap样式加载
# 【问题】默认CDN在中国可能被墙或慢
# 【解决】使用 cdn.staticfile.org（国内七牛云镜像）
bootstrap_css = "https://cdn.staticfile.org/twitter-bootstrap/5.3.2/css/bootstrap.min.css"

# 创建 Dash 应用实例
# 【关键参数解释】
app = dash.Dash(
    app_name,                              # 应用名称 - 用于内部识别
    external_stylesheets=[bootstrap_css],  # 外部CSS - 加载Bootstrap样式
    suppress_callback_exceptions=True      # 【重要】允许动态组件的回调
    # 为什么需要这个？
    # 我们的属性面板是动态生成的，定义回调时组件可能还不存在
    # 没有这个设置，Dash 启动会报错 "Component not found"
)

# ============================================================================
# 🔹 辅助函数：清理图表字典
# ============================================================================

def clean_figure_dict(fig_dict: Optional[Dict[str, Any]]) -> Dict[str, Any]:
    """
    确保图表字典有效，防止 None 值导致错误

    【使用场景】
    从客户端接收 dcc.Store 的数据时，可能为 None（首次加载）
    直接用 go.Figure(None) 会报错，需要先清理

    【设计思想】
    防御性编程 - 假设输入可能有问题，提前处理

    参数:
        fig_dict: Plotly 图表的字典表示，可能为 None

    返回:
        有效的图表字典，至少包含空的 data 和 layout

    示例:
        >>> clean_figure_dict(None)
        {"data": [], "layout": {}}
        >>> clean_figure_dict({"data": [...]})
        {"data": [...], "layout": {}}  # 原样返回
    """
    if fig_dict is None:
        return {"data": [], "layout": {}}  # 返回最小有效结构
    return fig_dict

# ============================================================================
# 🔹 验证安装成功 - 启动确认
# ============================================================================

print("✅ 库导入成功！")
print(f"📦 Dash 版本: {dash.__version__}")
print(f"📊 Plotly 版本: {plotly.__version__}")
print(f"🚀 应用名称: {app_name}")
print()
print("💡 提示: 如果看到此输出，说明所有依赖都已正确安装！")

✅ 库导入成功！
📦 Dash 版本: 2.18.2
📊 Plotly 版本: 5.24.1
🚀 应用名称: Interactive_Editor_262c8f50

💡 提示: 如果看到此输出，说明所有依赖都已正确安装！


---
## 1.2 核心数据模型：TraceDataset 类

### 🎯 设计思想

**问题场景：**
```python
# 没有统一数据模型时，信息散落各处
fig.data[0].x = [1,2,3]      # 数据在图表里
trace_color = "blue"          # 颜色在变量里
trace_name = "销售数据"        # 名称又在另一个变量
# 保存会话时怎么办？三个地方都要保存？😫
```

**解决方案：封装成类**
```python
# 用 TraceDataset 统一管理
dataset = TraceDataset(
    key="trace_1",
    name="销售数据",
    df=data,
    color="blue"
)
# 保存时只需要 dataset.to_dict()，一个对象搞定！
```

---

### 📊 类比理解

```
TraceDataset 就像 Photoshop 里的一个图层：
┌─────────────────────────────────────────┐
│  图层 1: "销售数据"                       │
│  ├── 数据内容 (df)     → 像素内容         │
│  ├── 颜色 (color)      → 图层颜色         │
│  ├── 线宽 (line_width) → 笔触粗细         │
│  ├── 点大小 (marker_size) → 笔刷大小      │
│  ├── 可见性 (visible)  → 显示/隐藏图层    │
│  └── 类型 (chart_type) → 图层混合模式     │
└─────────────────────────────────────────┘
```

---

### ❓ 可能的提问与回答

**Q1: 为什么用 `@dataclass` 而不是普通的 class？**
> **A:** `@dataclass` 是 Python 3.7+ 的语法糖，自动生成 `__init__`、`__repr__`、`__eq__` 等方法。比如：
> ```python
> # 普通类需要写
> class Dataset:
>     def __init__(self, key, name, df, color="#1f77b4"):
>         self.key = key
>         self.name = name
>         self.df = df
>         self.color = color
>     def __repr__(self):
>         return f"Dataset(key={self.key}, name={self.name})"
>
> # @dataclass 一行搞定
> @dataclass
> class Dataset:
>     key: str
>     name: str
>     df: pd.DataFrame
>     color: str = "#1f77b4"
> ```

**Q2: 为什么 `to_plotly_trace()` 方法要处理 x, y 不存在的情况？**
> **A:** 防御性编程！用户上传的 CSV 可能列名不是 'x' 和 'y'（比如 'time' 和 'value'）。我们需要优雅地处理这种情况，而不是直接崩溃。

**Q3: `visible = "legendonly"` 是什么意思？**
> **A:** Plotly 的可见性有三种状态：
> - `True` - 完全可见
> - `False` - 完全隐藏（包括图例）
> - `"legendonly"` - 图表中隐藏，但图例中保留（用户可以点击图例恢复）

**Q4: 为什么默认颜色是 `#1f77b4`？**
> **A:** 这是 Plotly 默认色板 `D3` 的第一个颜色（蓝色）。保持与 Plotly 默认行为一致，用户体验更统一。

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  Part 1.2: TraceDataset 数据容器类                                            ║
# ║  每个图表图层的数据和样式封装                                                    ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ============================================================================
# 【设计模式】数据传输对象 (DTO - Data Transfer Object)
#
# 什么是 DTO？
# - 一个只包含数据、不包含业务逻辑的简单对象
# - 用于在不同层之间传递数据
# - TraceDataset 就是我们自定义的 DTO
#
# 为什么需要 DTO？
# - 解耦：数据结构和业务逻辑分离
# - 序列化：方便转换为 JSON 保存
# - 类型安全：IDE 可以提供智能提示
# ============================================================================

@dataclass  # Python 3.7+ 的数据类装饰器
            # 自动生成 __init__, __repr__, __eq__ 等方法
            # 减少样板代码 (boilerplate code)
class TraceDataset:
    """
    图表图层的数据容器类

    ┌─────────────────────────────────────────────────────────────┐
    │  TraceDataset 的职责（单一职责原则）：                        │
    │  1. 存储一个图层的所有数据和样式                              │
    │  2. 提供转换为 Plotly trace 的方法                           │
    │  3. 不负责图层管理（那是 FigureStore 的工作）                 │
    └─────────────────────────────────────────────────────────────┘

    生命周期：
    创建 → 存入 FigureStore.datasets → 调用 to_plotly_trace() → 添加到 Figure

    属性说明:
        key: 唯一标识符，用于在字典中查找 (如 "trace_1", "trace_2")
        name: 显示名称，出现在图例中 (如 "销售数据 2024")
        df: 原始数据 DataFrame，至少包含可绘制的列
        color: 颜色 (十六进制如 "#1f77b4" 或颜色名如 "blue", "red")
        line_width: 线条宽度 (像素，建议 1-5)
        marker_size: 标记点大小 (像素，建议 4-15)
        visible: 是否可见 (True=显示, False=隐藏)
        chart_type: 图表类型 ("scatter", "bar", "pie", "histogram", "box")
    """

    # =========================================================================
    # 必需属性（没有默认值，创建时必须提供）
    # =========================================================================
    key: str
    # 唯一键 - 用于在字典中快速查找
    # 命名规范：使用 "trace_1", "trace_2" 或描述性名称 "sales_2024"
    # 【重要】key 一旦设定不应更改，因为其他地方可能引用它

    name: str
    # 显示名称 - 出现在图例 (legend) 中
    # 命名规范：使用用户友好的名称，如 "销售数据", "温度曲线"
    # 【提示】name 可以包含中文、空格等，更灵活

    df: pd.DataFrame
    # 原始数据 - 存储实际的数值数据
    # 【结构要求】通常需要 'x' 和 'y' 列，但代码会智能处理其他情况
    # 【内存考虑】DataFrame 可能很大，避免不必要的复制

    # =========================================================================
    # 可选属性（有默认值，可以省略）
    # =========================================================================
    color: str = "#1f77b4"
    # 颜色 - 支持多种格式：
    # - 十六进制: "#1f77b4", "#ff0000"
    # - 颜色名: "blue", "red", "green"
    # - RGB: "rgb(255, 0, 0)"
    # - RGBA: "rgba(255, 0, 0, 0.5)"
    # 【默认值】#1f77b4 是 Plotly D3 色板的第一个颜色（蓝色）

    line_width: float = 2.5
    # 线宽 - 单位是像素
    # 【建议范围】1.0 - 5.0，太细看不清，太粗太丑
    # 【适用】line, scatter (mode包含lines)

    marker_size: float = 6.0
    # 点大小 - 单位是像素
    # 【建议范围】4 - 15，根据数据密度调整
    # 【适用】scatter, bubble (bubble 可以更大)

    visible: bool = True
    # 可见性 - 控制图层显示/隐藏
    # True: 正常显示
    # False: 隐藏（转换时变成 "legendonly"）
    # 【用途】用户可以通过点击图例来切换

    chart_type: str = "scatter"
    # 图表类型 - 决定使用哪种 Plotly trace
    # 支持的类型: "scatter", "bar", "pie", "histogram", "box"
    # 【扩展点】要添加新图表类型，修改 to_plotly_trace() 方法

    def to_plotly_trace(self):
        """
        将 TraceDataset 转换为 Plotly 的 Trace 对象

        【核心方法】这是 TraceDataset 最重要的方法！
        它实现了从"我们的数据模型"到"Plotly能理解的格式"的转换

        ┌─────────────────────────────────────────────────────────────┐
        │  转换流程：                                                  │
        │  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐  │
        │  │ TraceDataset │ →  │ to_plotly_   │ →  │ go.Scatter   │  │
        │  │ (我们的格式) │    │ trace()      │    │ (Plotly格式) │  │
        │  └──────────────┘    └──────────────┘    └──────────────┘  │
        └─────────────────────────────────────────────────────────────┘

        返回:
            go.Scatter / go.Bar / go.Pie 等 Plotly trace 对象

        错误处理:
            如果数据不足，返回一个空的 Scatter（带错误提示名称）
        """

        # =====================================================================
        # 步骤1: 安全地获取 x, y 数据
        # =====================================================================
        # 【问题】用户的 CSV 列名可能不是 'x' 和 'y'
        # 【解决】先尝试标准列名，失败则用前两列

        # 检查 'x' 列是否存在
        x = self.df['x'] if 'x' in self.df.columns else None
        # 检查 'y' 列是否存在
        y = self.df['y'] if 'y' in self.df.columns else None

        # 如果标准列名不存在，尝试使用前两列作为备选
        if x is None or y is None:
            if self.df.shape[1] >= 2:  # 至少有2列才能绘图
                x = self.df.iloc[:, 0]  # 第一列作为 x
                y = self.df.iloc[:, 1]  # 第二列作为 y
                # 【注意】iloc 是位置索引，不受列名影响
            else:
                # 数据确实不足，返回带错误提示的空 trace
                return go.Scatter(name=f"{self.name} (Empty)")

        # =====================================================================
        # 步骤2: 根据图表类型创建对应的 trace
        # =====================================================================
        # 【设计模式】策略模式 (Strategy Pattern) 的简化版
        # 根据 chart_type 选择不同的创建策略

        if self.chart_type == "bar":
            # ━━━━━ 柱状图 ━━━━━
            # 适用场景: 分类数据比较
            # x: 类别标签 (如月份、产品名)
            # y: 数值 (如销售额、数量)
            trace = go.Bar(
                x=x,           # 类别轴
                y=y,           # 数值轴
                name=self.name # 图例名称
            )

        elif self.chart_type == "pie":
            # ━━━━━ 饼图 ━━━━━
            # 适用场景: 展示比例分布
            # 【注意】饼图参数不同！用 labels 和 values，不是 x 和 y
            trace = go.Pie(
                labels=x,      # 扇区标签
                values=y,      # 扇区数值（自动计算百分比）
                name=self.name
            )

        elif self.chart_type == "histogram":
            # ━━━━━ 直方图 ━━━━━
            # 适用场景: 展示数值分布
            # 【注意】直方图只需要一个数据序列
            trace = go.Histogram(
                x=y,           # 用 y 数据，因为这通常是我们想看分布的列
                name=self.name
            )

        elif self.chart_type == "box":
            # ━━━━━ 箱线图 ━━━━━
            # 适用场景: 展示数据分布统计特征 (四分位数、异常值)
            trace = go.Box(
                y=y,           # 数值数据
                name=self.name
            )

        else:
            # ━━━━━ 默认：散点图 + 连线 ━━━━━
            # 最通用的图表类型
            trace = go.Scatter(
                x=x,
                y=y,
                mode="lines+markers",  # 同时显示线和点
                # 【可选值】"lines", "markers", "lines+markers", "none"
                name=self.name,
            )

        # =====================================================================
        # 步骤3: 应用样式设置
        # =====================================================================
        # 【设计】使用字典收集更新，然后一次性应用
        # 这比多次调用 trace.update() 更高效

        updates = {}

        # 检查 trace 是否支持 marker（不是所有图表类型都有）
        if hasattr(trace, "marker"):
            updates["marker"] = dict(
                size=self.marker_size,  # 点的大小
                color=self.color        # 点的颜色
            )

        # 检查 trace 是否支持 line
        if hasattr(trace, "line"):
            updates["line"] = dict(
                width=self.line_width,  # 线的宽度
                color=self.color        # 线的颜色
            )

        # 应用所有样式更新
        if updates:
            trace.update(**updates)  # **updates 解包字典为关键字参数

        # =====================================================================
        # 步骤4: 设置可见性
        # =====================================================================
        # Plotly 的可见性是个特殊属性，有三种值：
        # - True: 正常显示
        # - False: 完全隐藏（包括图例）
        # - "legendonly": 图表隐藏，图例保留（用户可点击恢复）
        #
        # 我们选择 "legendonly" 作为"隐藏"状态，因为：
        # 1. 用户可以点击图例快速恢复
        # 2. 视觉上更清晰地表示"这个数据存在但被隐藏"

        trace.visible = True if self.visible else "legendonly"

        return trace


# ============================================================================
# 🔹 演示：创建一个 TraceDataset 实例并查看效果
# ============================================================================

print("=" * 60)
print("📊 TraceDataset 演示")
print("=" * 60)

# 创建测试数据
demo_df = pd.DataFrame({
    'x': [1, 2, 3, 4, 5],
    'y': [2, 4, 1, 5, 3]
})
print(f"\n1️⃣ 创建测试数据:")
print(demo_df.to_string(index=False))

# 创建 TraceDataset 实例
demo_trace = TraceDataset(
    key="demo_1",           # 唯一标识符
    name="演示数据",         # 显示名称
    df=demo_df,             # 数据
    color="red",            # 红色
    marker_size=10          # 较大的点
)

print(f"\n2️⃣ 创建 TraceDataset:")
print(f"   key: {demo_trace.key}")
print(f"   name: {demo_trace.name}")
print(f"   color: {demo_trace.color}")
print(f"   marker_size: {demo_trace.marker_size}")
print(f"   chart_type: {demo_trace.chart_type}")
print(f"   visible: {demo_trace.visible}")

# 转换为 Plotly trace
plotly_trace = demo_trace.to_plotly_trace()
print(f"\n3️⃣ 转换为 Plotly trace:")
print(f"   类型: {type(plotly_trace).__name__}")
print(f"   mode: {getattr(plotly_trace, 'mode', 'N/A')}")
print(f"   marker.color: {plotly_trace.marker.color if hasattr(plotly_trace, 'marker') else 'N/A'}")

print("\n✅ TraceDataset 演示完成！")

📊 TraceDataset 演示

1️⃣ 创建测试数据:
 x  y
 1  2
 2  4
 3  1
 4  5
 5  3

2️⃣ 创建 TraceDataset:
   key: demo_1
   name: 演示数据
   color: red
   marker_size: 10
   chart_type: scatter
   visible: True

3️⃣ 转换为 Plotly trace:
   类型: Scatter
   mode: lines+markers
   marker.color: red

✅ TraceDataset 演示完成！


---
# 🧠 Part 2: FigureStore 核心状态管理
## 2.1 为什么需要状态管理？

---

### 🎯 问题场景

```python
# ❌ 没有状态管理的混乱情况
fig = go.Figure()                    # 图表对象
datasets = {"trace1": df1}           # 数据集字典
colors = {"trace1": "blue"}          # 颜色配置
history = []                         # 历史记录
metadata = {"created": "2024-01-01"} # 元信息

# 问题1: 修改图表时，多个变量需要同步更新
fig.data[0].marker.color = "red"
colors["trace1"] = "red"  # 容易忘记！

# 问题2: 保存会话需要手动收集所有变量
save_data = {
    "fig": fig.to_dict(),
    "datasets": datasets,
    "colors": colors,
    ...  # 越来越多！😫
}
```

---

### ✅ 解决方案：FigureStore 集中管理

```
┌─────────────────────────────────────────────────────────────────┐
│                     FigureStore (单一数据源)                      │
│                                                                  │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐           │
│  │   figure     │  │   datasets   │  │ data_repo    │           │
│  │  (当前图表)   │  │ (所有图层)   │  │ (原始数据)   │           │
│  └──────────────┘  └──────────────┘  └──────────────┘           │
│                                                                  │
│  ┌──────────────┐  ┌──────────────┐                             │
│  │   metadata   │  │ dataset_order│                             │
│  │  (元信息)    │  │ (显示顺序)   │                             │
│  └──────────────┘  └──────────────┘                             │
│                                                                  │
│  方法: add_dataset() | rebuild_figure() | serialize() | load()  │
└─────────────────────────────────────────────────────────────────┘
                              ↕
              所有 Callback 都操作这一个实例
```

---

### 📚 设计模式解析

**1. 单例模式 (Singleton)**
```python
# 全局只有一个 FigureStore 实例
figure_store = FigureStore()  # 在模块级别创建
# 所有 Callback 都使用同一个 figure_store
```

**2. 仓库模式 (Repository Pattern)**
```python
# FigureStore 像一个"仓库"，统一管理数据
figure_store.add_dataset(...)      # 添加
figure_store.datasets.get(key)     # 查询
figure_store.remove_trace(index)   # 删除
```

**3. 命令模式 (Command Pattern) 的影子**
```python
# 所有修改都通过方法进行，而不是直接操作属性
figure_store.update_figure(fig)    # ✅ 正确
figure_store.figure = fig          # ❌ 避免（虽然可行）
```

---

### ❓ 可能的提问与回答

**Q1: 为什么不直接操作 `go.Figure`？**
> **A:** 因为我们需要在修改图表的同时：
> - 更新元信息 (updated_at)
> - 保持数据集同步 (datasets)
> - 支持撤销功能 (需要保存完整状态)
> 直接操作 Figure 会导致这些额外工作被遗漏。

**Q2: `data_repository` 和 `datasets` 有什么区别？**
> **A:**
> - `data_repository`: 存储**原始** DataFrame（用户上传的 CSV）
> - `datasets`: 存储**处理后**的 TraceDataset（包含样式信息）
>
> 类比：`data_repository` 是"食材仓库"，`datasets` 是"已经调味准备下锅的菜"。

**Q3: 为什么 `rebuild_figure_from_datasets()` 要重新创建 Figure？**
> **A:** 因为 Plotly 的 Figure 是不可变风格的。修改数据集后，最安全的方式是重新构建整个 Figure，而不是尝试修改现有的 trace。这避免了很多边界情况的 bug。

**Q4: `_touch()` 方法的作用是什么？**
> **A:** 它更新 `metadata["updated_at"]` 时间戳。每次状态改变时调用，用于：
> - 追踪最后修改时间
> - 保存会话时记录版本
> - 调试时查看状态变化

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  Part 2.1: FigureStore 核心状态管理类                                          ║
# ║  整个应用的"大脑" - 管理图表、数据、元信息                                        ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

class FigureStore:
    """
    图表状态管理器 - 整个应用的核心类

    职责:
        1. 存储当前 Plotly 图表对象
        2. 管理所有数据集 (TraceDataset)
        3. 维护数据仓库 (原始 DataFrame)
        4. 提供序列化/反序列化接口 (保存/加载会话)

    设计思想:
        - 集中式状态管理 (类似 Redux 的 Store)
        - 单一数据源 (Single Source of Truth)
        - 所有修改通过方法进行，保证一致性
    """

    def __init__(self, theme: str = "plotly_white") -> None:
        """
        初始化 FigureStore

        参数:
            theme: Plotly 主题，如 "plotly_white", "plotly_dark", "ggplot2" 等
        """
        # ====== 主要状态属性 ======
        self.current_theme: str = theme          # 当前主题
        self.figure: Optional[go.Figure] = None  # 当前 Plotly 图表对象
        self.datasets: Dict[str, TraceDataset] = {}  # 所有数据集 {key: TraceDataset}
        self.dataset_order: List[str] = []       # 数据集显示顺序 (图层顺序)

        # ====== 数据仓库 ======
        # 存储用户导入的原始 DataFrame
        # 与 datasets 的区别：这里存原始数据，datasets 存处理后的图层数据
        self.data_repository: Dict[str, pd.DataFrame] = {}

        # ====== 元信息 ======
        self.metadata: Dict[str, Any] = {
            "created_at": datetime.now().isoformat(timespec="seconds"),  # 创建时间
            "updated_at": None,   # 最后更新时间
            "version": "1.0.0",   # 版本号
        }

        # 初始化时创建默认演示图表
        self._init_default_figure()

    # =========================================================================
    # 🔹 初始化方法
    # =========================================================================

    def _init_default_figure(self) -> None:
        """
        创建默认演示图表 - 阻尼正弦波

        数学公式: y = sin(t) * exp(-0.15 * t)
        这是一个经典的物理现象：阻尼振动

        为什么选这个图形？
        1. 视觉上有趣，容易解释
        2. 包含足够多的数据点，适合演示交互
        3. 有 x, y, z 三维数据，支持 3D 图表
        """
        # 生成时间序列 (0 到 10，共 200 个点)
        t = np.linspace(0, 10, 200)

        # 阻尼正弦波: 振幅随时间指数衰减
        signal = np.sin(t) * np.exp(-0.15 * t)

        # 添加 Z 轴数据，支持 3D 图表演示
        z_val = np.cos(t) * t

        # 创建 DataFrame
        df = pd.DataFrame({"x": t, "y": signal, "z": z_val})

        # 添加到数据仓库（原始数据备份）
        self.add_dataframe("demo_signal", df)

        # 添加为第一个数据集
        self.add_dataset(
            key="trace_1",       # 唯一标识符
            name="Demo Signal",  # 显示名称
            df=df,               # 数据
            color="#1f77b4",     # 蓝色
        )

        # 从数据集重建图表
        self.rebuild_figure_from_datasets()

    # =========================================================================
    # 🔹 数据仓库管理方法
    # =========================================================================

    def add_dataframe(self, name: str, df: pd.DataFrame) -> None:
        """
        将 DataFrame 添加到数据仓库

        参数:
            name: 数据集名称
            df: Pandas DataFrame

        用途:
            - 保存用户上传的 CSV 文件
            - 保存生成的数据
        """
        if name is None:
            return
        # 清理名称，替换特殊字符
        safe_name = name.replace(" ", "_").replace("-", "_").replace(".", "_")
        self.data_repository[safe_name] = df
        self._touch()  # 更新时间戳

    def get_dataframe(self, name: str) -> Optional[pd.DataFrame]:
        """获取数据仓库中的 DataFrame"""
        return self.data_repository.get(name)

    def _touch(self) -> None:
        """更新 metadata 中的最后更新时间"""
        self.metadata["updated_at"] = datetime.now().isoformat(timespec="seconds")

    # =========================================================================
    # 🔹 布局配置方法
    # =========================================================================

    def _base_layout(self) -> Dict[str, Any]:
        """
        基础布局配置 - 类似 MATLAB 的默认图表外观

        返回:
            布局字典，包含主题、边距、图例等设置
        """
        layout: Dict[str, Any] = {
            "template": self.current_theme,  # 主题模板
            "margin": dict(l=60, r=20, t=50, b=60),  # 左右上下边距
            "hovermode": "closest",  # 悬停模式：显示最近的点
            "legend": dict(
                orientation="h",      # 水平布局
                yanchor="bottom",     # 底部对齐
                y=1.02,               # 在图表上方
                xanchor="right",      # 右对齐
                x=1.0,
            ),
        }
        return layout

    def update_theme(self, theme: str) -> None:
        """更新图表主题"""
        self.current_theme = theme or self.current_theme
        if self.figure is not None:
            self.figure.update_layout(template=self.current_theme)
            self._touch()

    # =========================================================================
    # 🔹 图表操作方法
    # =========================================================================

    def get_figure_dict(self) -> Optional[Dict[str, Any]]:
        """
        获取图表的字典表示 (用于序列化)

        为什么需要字典？
        1. Dash 的 dcc.Graph 接受字典格式
        2. 方便 JSON 序列化保存
        3. 可以直接传递给前端
        """
        return self.figure.to_dict() if self.figure is not None else None

    def update_figure(self, fig: go.Figure) -> None:
        """
        替换当前图表

        使用场景:
        - 用户通过代码创建新图表
        - 通过交互编辑修改后更新
        """
        self.figure = fig
        self._touch()

    # =========================================================================
    # 🔹 数据集管理方法 (核心!)
    # =========================================================================

    def add_dataset(
        self,
        key: str,
        name: str,
        df: pd.DataFrame,
        color: str = "#1f77b4",
        line_width: float = 2.5,
        marker_size: float = 6.0,
        visible: bool = True,
        chart_type: str = "scatter",
    ) -> None:
        """
        添加新数据集

        参数:
            key: 唯一标识符 (如 "trace_1")
            name: 显示名称 (如 "销售数据")
            df: 数据 DataFrame
            color: 颜色
            line_width: 线宽
            marker_size: 点大小
            visible: 是否可见
            chart_type: 图表类型

        流程:
            1. 重置 DataFrame 索引
            2. 创建 TraceDataset 实例
            3. 存入 datasets 字典
            4. 添加到显示顺序列表
        """
        df = df.reset_index(drop=True)  # 重置索引，避免索引问题

        # 创建 TraceDataset 实例
        dataset = TraceDataset(
            key=key,
            name=name,
            df=df,
            color=color,
            line_width=line_width,
            marker_size=marker_size,
            visible=visible,
            chart_type=chart_type,
        )

        # 存入字典
        self.datasets[key] = dataset

        # 添加到顺序列表（如果还没有的话）
        if key not in self.dataset_order:
            self.dataset_order.append(key)

    def rebuild_figure_from_datasets(self) -> None:
        """
        从所有数据集重建 Plotly 图表

        这是非常重要的方法！

        工作流程:
            1. 创建空的 Figure
            2. 按顺序遍历所有数据集
            3. 将每个数据集转换为 trace 并添加
            4. 应用基础布局
            5. 保留现有的标题设置
            6. 更新 self.figure
        """
        # 步骤1: 创建空图表
        fig = go.Figure()

        # 步骤2-3: 按顺序添加所有 trace
        for key in self.dataset_order:
            dataset = self.datasets.get(key)
            if not dataset:
                continue  # 跳过不存在的数据集

            # 调用 TraceDataset 的方法转换为 Plotly trace
            trace = dataset.to_plotly_trace()
            fig.add_trace(trace)

        # 步骤4: 应用基础布局
        fig.update_layout(**self._base_layout())

        # 步骤5: 保留现有标题
        if self.figure is not None:
            layout = getattr(self.figure, 'layout', None)
            if layout:
                existing_title = getattr(layout, 'title', None)
                if existing_title and getattr(existing_title, 'text', None):
                    fig.update_layout(title=existing_title)

        # 步骤6: 更新 self.figure
        self.update_figure(fig)

    def set_trace_visibility(self, visible_keys: List[str]) -> None:
        """设置哪些数据集可见"""
        visible_set = set(visible_keys or [])
        for key, dataset in self.datasets.items():
            dataset.visible = key in visible_set
        self._touch()

    def remove_trace(self, index: int) -> None:
        """
        按索引删除 trace

        参数:
            index: trace 在图表中的索引 (从 0 开始)

        流程:
            1. 从 figure.data 中删除
            2. 从 datasets 字典中删除
            3. 从 dataset_order 列表中删除
        """
        if self.figure is None:
            return

        # 从图表数据中删除
        data = list(self.figure.data)
        if 0 <= index < len(data):
            data.pop(index)
            self.figure.data = tuple(data)
            self._touch()

        # 从内部数据结构中删除
        if index < len(self.dataset_order):
            key = self.dataset_order[index]
            if key in self.datasets:
                del self.datasets[key]
            self.dataset_order.pop(index)

    def remove_annotation(self, index: int) -> None:
        """按索引删除注释"""
        if self.figure is None:
            return

        layout = cast(Any, self.figure.layout)
        if not layout or not hasattr(layout, 'annotations'):
            return

        annots = list(layout.annotations) if layout.annotations else []
        if 0 <= index < len(annots):
            annots.pop(index)
            self.figure.update_layout(annotations=annots)
            self._touch()

    def remove_shape(self, index: int) -> None:
        """按索引删除形状"""
        if self.figure is None:
            return
        layout = cast(Any, self.figure.layout)
        if not layout or not hasattr(layout, 'shapes'):
            return
        shapes = list(layout.shapes) if layout.shapes else []
        if 0 <= index < len(shapes):
            shapes.pop(index)
            self.figure.update_layout(shapes=shapes)
            self._touch()

    # =========================================================================
    # 🔹 序列化方法 (保存/加载会话)
    # =========================================================================

    def serialize_session(self) -> Dict[str, Any]:
        """
        序列化当前会话为字典 (可转为 JSON 保存)

        返回:
            包含图表、数据、元信息的完整会话字典

        结构:
        {
            "figure": {...},           # Plotly 图表字典
            "datasets": [...],         # 所有数据集
            "data_repository": {...},  # 原始数据仓库
            "dataset_order": [...],    # 显示顺序
            "metadata": {...}          # 元信息
        }
        """
        serialized_datasets = []
        for key in self.dataset_order:
            ds = self.datasets.get(key)
            if ds is None:
                continue
            # 将 DataFrame 转为可序列化的字典格式
            serialized_datasets.append({
                "key": ds.key,
                "name": ds.name,
                "df": ds.df.to_dict(orient="split"),  # split 格式更紧凑
                "color": ds.color,
                "line_width": ds.line_width,
                "marker_size": ds.marker_size,
                "visible": ds.visible,
                "chart_type": ds.chart_type,
            })

        # 序列化数据仓库
        repo_serialized = {}
        for name, df in self.data_repository.items():
            repo_serialized[name] = df.to_dict(orient="split")

        return {
            "figure": self.figure.to_dict() if self.figure else None,
            "datasets": serialized_datasets,
            "data_repository": repo_serialized,
            "dataset_order": self.dataset_order,
            "metadata": self.metadata,
        }

    def load_session(self, payload: Dict[str, Any]) -> None:
        """
        从序列化的字典加载会话

        参数:
            payload: serialize_session() 返回的字典

        流程:
            1. 清空现有数据
            2. 恢复数据仓库
            3. 恢复数据集
            4. 恢复图表
            5. 更新元信息
        """
        # 清空现有数据
        self.datasets.clear()
        self.dataset_order.clear()
        self.data_repository.clear()

        # 恢复数据仓库
        for name, df_dict in payload.get("data_repository", {}).items():
            self.data_repository[name] = pd.DataFrame(**df_dict)

        # 恢复数据集
        for ds_dict in payload.get("datasets", []):
            df = pd.DataFrame(**ds_dict["df"])
            self.add_dataset(
                key=ds_dict["key"],
                name=ds_dict["name"],
                df=df,
                color=ds_dict.get("color", "#1f77b4"),
                line_width=ds_dict.get("line_width", 2.5),
                marker_size=ds_dict.get("marker_size", 6.0),
                visible=ds_dict.get("visible", True),
                chart_type=ds_dict.get("chart_type", "scatter"),
            )

        # 恢复数据集顺序
        self.dataset_order = payload.get("dataset_order", list(self.datasets.keys()))

        # 恢复图表
        fig_data = payload.get("figure")
        if fig_data:
            self.figure = go.Figure(fig_data)
        else:
            self.rebuild_figure_from_datasets()

        # 恢复元信息
        self.metadata.update(payload.get("metadata", {}))
        self._touch()


# ============================================================================
# 🔹 演示：创建 FigureStore 实例并查看状态
# ============================================================================

# 创建 FigureStore 实例
demo_store = FigureStore(theme="plotly_white")

print("✅ FigureStore 创建成功!")
print(f"   主题: {demo_store.current_theme}")
print(f"   数据集数量: {len(demo_store.datasets)}")
print(f"   数据仓库数量: {len(demo_store.data_repository)}")
print(f"   创建时间: {demo_store.metadata['created_at']}")

# 显示数据集信息
print("\n📊 数据集列表:")
for key, ds in demo_store.datasets.items():
    print(f"   - {key}: {ds.name} ({len(ds.df)} 行, 类型: {ds.chart_type})")

✅ FigureStore 创建成功!
   主题: plotly_white
   数据集数量: 1
   数据仓库数量: 1
   创建时间: 2025-12-03T09:09:22

📊 数据集列表:
   - trace_1: Demo Signal (200 行, 类型: scatter)


---
# ⏪ Part 3: 历史系统与代码生成
## 3.1 HistoryStack - 撤销/重做功能

---

### 🎯 经典数据结构：双栈实现 Undo/Redo

```
【可视化理解】

操作序列: 画圆 → 改蓝 → 加线 → 改红
                              ↑ 当前状态

Undo Stack (撤销栈):           Redo Stack (重做栈):
┌─────────┐                   ┌─────────┐
│  改红   │ ← 栈顶 (当前)      │  (空)   │
├─────────┤                   └─────────┘
│  加线   │
├─────────┤
│  改蓝   │
├─────────┤
│  画圆   │ ← 栈底 (最初)
└─────────┘

--- 用户点击 Undo ---

Undo Stack:                   Redo Stack:
┌─────────┐                   ┌─────────┐
│  加线   │ ← 现在是当前       │  改红   │ ← 可以 Redo
├─────────┤                   └─────────┘
│  改蓝   │
├─────────┤
│  画圆   │
└─────────┘

--- 用户点击 Redo ---

Undo Stack:                   Redo Stack:
┌─────────┐                   ┌─────────┐
│  改红   │ ← 恢复了！         │  (空)   │
├─────────┤                   └─────────┘
│  加线   │
├─────────┤
│  改蓝   │
├─────────┤
│  画圆   │
└─────────┘
```

---

### ❓ 可能的提问与回答

**Q1: 为什么用两个栈而不是一个？**
> **A:** 一个栈只能实现 Undo。要支持 Redo，需要第二个栈来保存"被撤销的状态"。当用户 Undo 时，当前状态移到 Redo 栈；Redo 时再移回来。

**Q2: `max_size=50` 是怎么考虑的？**
> **A:** 每个状态都是完整图表的深拷贝，内存消耗不小。50 个状态是平衡内存占用和用户体验的经验值。超过 50 个时，删除最早的状态。

**Q3: 为什么要检查 "duplicate states"（重复状态）？**
> **A:** 有些操作可能不改变图表（如点击取消），如果每次都保存，栈会被无意义的重复状态填满。通过 JSON 比较来去重。

**Q4: 为什么用 `copy.deepcopy()` 而不是普通赋值？**
> **A:** Python 的赋值是引用复制！
> ```python
> a = {"x": [1, 2, 3]}
> b = a           # b 和 a 指向同一个对象
> b["x"][0] = 99  # a 也被改了！
>
> c = copy.deepcopy(a)  # c 是完全独立的副本
> c["x"][0] = 0         # a 不受影响
> ```

---

## 3.2 CodeGenerator - 代码生成器

### 🎯 设计目标：所见即所得 + 所得即代码

```
传统流程:
用户想要图表 → 写代码 → 看效果 → 不满意 → 改代码 → 看效果 → ...

我们的流程:
用户交互编辑 → 满意效果 → 一键生成代码 → 代码可复现图表！
```

---

### ❓ 可能的提问与回答

**Q1: 生成的代码是什么格式？**
> **A:** 我们生成的是完整、可运行的 Python 脚本：
> ```python
> import json
> import plotly.graph_objects as go
>
> fig_dict = json.loads('...')  # 序列化的图表数据
> fig = go.Figure(fig_dict)
> fig.show()
> ```

**Q2: 为什么用 `fig.to_json()` 而不是保存代码调用？**
> **A:** 因为用户的修改可能非常复杂（颜色、注释、形状...），要追踪所有操作并生成对应代码太复杂。直接序列化整个图表是最可靠的方式。

**Q3: `generate_smart_plot_code()` 方法的作用？**
> **A:** 这个方法更智能！它分析 DataFrame 的列类型，自动选择合适的列作为 x, y, color 等参数，生成的代码更加"人类可读"：
> ```python
> # 自动生成的代码
> fig = px.scatter(df, x='日期', y='销售额', color='区域')
> ```

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  Part 3.1: HistoryStack - 撤销/重做系统                                        ║
# ║  使用双栈实现经典的 Undo/Redo 功能                                              ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ============================================================================
# 【数据结构】栈 (Stack) - LIFO (后进先出)
#
# 为什么用栈实现 Undo/Redo？
# - Undo 总是撤销"最近的操作"（后进先出）
# - 这正是栈的特性！
#
# 为什么需要两个栈？
# - Undo Stack: 存储历史状态，支持撤销
# - Redo Stack: 存储被撤销的状态，支持重做
# ============================================================================

class HistoryStack:
    """
    经典的 Undo/Redo 双栈实现

    ┌─────────────────────────────────────────────────────────────────┐
    │  工作原理：                                                      │
    │                                                                  │
    │  push(state):  当前状态 → Undo栈顶                               │
    │  undo():       Undo栈顶 → Redo栈顶，返回新的Undo栈顶              │
    │  redo():       Redo栈顶 → Undo栈顶，返回该状态                    │
    │                                                                  │
    │  【关键】每次 push 新状态时，Redo栈会被清空！                      │
    │  因为新操作意味着"另一条时间线"，之前撤销的内容不再有效             │
    └─────────────────────────────────────────────────────────────────┘
    """

    def __init__(self, max_size: int = 50) -> None:
        """
        初始化历史栈

        参数:
            max_size: 最大保存的状态数量
                     【权衡】太大占内存，太小用户体验差
                     50 是经过测试的合理值
        """
        self.max_size = max_size

        # 使用 Python List 作为栈（append/pop 操作栈顶）
        self.undo_stack: List[Dict[str, Any]] = []  # 撤销栈
        self.redo_stack: List[Dict[str, Any]] = []  # 重做栈

    def push(self, fig_dict: Optional[Dict[str, Any]]) -> None:
        """
        保存当前状态到历史栈

        【调用时机】
        - 用户执行任何修改操作后（改颜色、添加注释等）
        - 不是每次回调都调用，只有"有意义的改变"才保存

        参数:
            fig_dict: 当前图表的字典表示 (fig.to_dict())
        """
        # 防御性检查：空值不保存
        if fig_dict is None:
            return

        # 【关键】必须深拷贝！否则后续修改会影响历史记录
        snapshot = copy.deepcopy(fig_dict)

        # ━━━━━ 去重检查 ━━━━━
        # 【问题】有些操作可能不改变图表（如取消操作）
        # 【解决】比较新状态和最后一个状态，相同则不保存
        if self.undo_stack:
            try:
                # 使用 JSON 序列化进行比较（忽略对象地址差异）
                last_state = json.dumps(self.undo_stack[-1], sort_keys=True, default=str)
                new_state = json.dumps(snapshot, sort_keys=True, default=str)
                if last_state == new_state:
                    return  # 状态相同，不保存
            except Exception:
                pass  # 比较失败就照常保存

        # 添加到撤销栈
        self.undo_stack.append(snapshot)

        # 限制栈大小（删除最老的记录）
        if len(self.undo_stack) > self.max_size:
            self.undo_stack.pop(0)  # pop(0) 删除栈底（最老的）

        # 【关键】新操作清空重做栈！
        # 逻辑：用户做了新操作，之前撤销的"未来"不再有效
        self.redo_stack.clear()

    def can_undo(self) -> bool:
        """
        是否可以撤销

        【注意】至少需要2个状态才能撤销（当前状态 + 至少1个历史状态）
        只有1个状态时，那就是初始状态，无法再撤销
        """
        return len(self.undo_stack) > 1

    def can_redo(self) -> bool:
        """是否可以重做（Redo栈非空即可）"""
        return bool(self.redo_stack)

    def undo(self) -> Optional[Dict[str, Any]]:
        """
        执行撤销操作

        返回:
            撤销后应该显示的状态，如果无法撤销则返回 None

        【状态流转】
        Undo栈: [A, B, C]  →  Undo栈: [A, B]
        Redo栈: []         →  Redo栈: [C]
                               ↑
                              返回 B
        """
        if not self.can_undo():
            return None

        # 当前状态（栈顶）移到 Redo 栈
        current = self.undo_stack.pop()
        self.redo_stack.append(current)

        # 返回新的栈顶（上一个状态）的深拷贝
        return copy.deepcopy(self.undo_stack[-1])

    def redo(self) -> Optional[Dict[str, Any]]:
        """
        执行重做操作

        返回:
            重做后应该显示的状态，如果无法重做则返回 None

        【状态流转】
        Undo栈: [A, B]     →  Undo栈: [A, B, C]
        Redo栈: [C]        →  Redo栈: []
                               ↑
                              返回 C
        """
        if not self.redo_stack:
            return None

        # 从 Redo 栈取出状态
        state = self.redo_stack.pop()

        # 添加到 Undo 栈
        self.undo_stack.append(copy.deepcopy(state))

        # 返回该状态的深拷贝
        return copy.deepcopy(state)


# ============================================================================
# 🔹 ActionLog - 操作日志（用于调试和分析）
# ============================================================================

class ActionLog:
    """
    操作日志 - 记录用户的高级操作

    【区别于 HistoryStack】
    - HistoryStack: 保存完整状态，用于 Undo/Redo
    - ActionLog: 只保存操作类型和参数，用于调试/分析

    【用途】
    - 调试：查看用户做了什么导致问题
    - 分析：统计最常用的功能
    - 回放：理论上可以重放操作（未实现）
    """

    def __init__(self, max_actions: int = 500) -> None:
        """
        初始化操作日志

        参数:
            max_actions: 最多保存的操作数量
        """
        self.max_actions = max_actions
        self.actions: List[Dict[str, Any]] = []

    def record(self, action_type: str, payload: Optional[Dict[str, Any]] = None) -> None:
        """
        记录一个操作

        参数:
            action_type: 操作类型 (如 "add_trace", "change_color", "undo")
            payload: 操作的详细参数

        示例:
            action_log.record("change_color", {"trace_id": 0, "new_color": "red"})
        """
        entry = {
            "type": action_type,
            "payload": copy.deepcopy(payload or {}),
            "timestamp": datetime.now().isoformat(timespec="seconds"),
        }
        self.actions.append(entry)

        # 限制大小
        if len(self.actions) > self.max_actions:
            self.actions.pop(0)


# ============================================================================
# 🔹 演示：HistoryStack 的工作流程
# ============================================================================

print("=" * 60)
print("⏪ HistoryStack 演示")
print("=" * 60)

# 创建测试用的历史栈
demo_history = HistoryStack(max_size=5)

# 模拟用户操作
states = [
    {"data": "状态A - 初始"},
    {"data": "状态B - 改了颜色"},
    {"data": "状态C - 加了注释"},
    {"data": "状态D - 又改了大小"},
]

print("\n1️⃣ 模拟用户操作，依次 push 4个状态:")
for i, state in enumerate(states):
    demo_history.push(state)
    print(f"   push({state['data']})")
    print(f"   → Undo栈大小: {len(demo_history.undo_stack)}, Redo栈大小: {len(demo_history.redo_stack)}")

print(f"\n2️⃣ 当前可以 Undo: {demo_history.can_undo()}, 可以 Redo: {demo_history.can_redo()}")

print("\n3️⃣ 执行两次 Undo:")
for i in range(2):
    result = demo_history.undo()
    print(f"   undo() → 返回: {result['data'] if result else 'None'}")
    print(f"   → Undo栈大小: {len(demo_history.undo_stack)}, Redo栈大小: {len(demo_history.redo_stack)}")

print(f"\n4️⃣ 现在可以 Redo: {demo_history.can_redo()}")

print("\n5️⃣ 执行一次 Redo:")
result = demo_history.redo()
print(f"   redo() → 返回: {result['data'] if result else 'None'}")
print(f"   → Undo栈大小: {len(demo_history.undo_stack)}, Redo栈大小: {len(demo_history.redo_stack)}")

print("\n✅ HistoryStack 演示完成！")

⏪ HistoryStack 演示

1️⃣ 模拟用户操作，依次 push 4个状态:
   push(状态A - 初始)
   → Undo栈大小: 1, Redo栈大小: 0
   push(状态B - 改了颜色)
   → Undo栈大小: 2, Redo栈大小: 0
   push(状态C - 加了注释)
   → Undo栈大小: 3, Redo栈大小: 0
   push(状态D - 又改了大小)
   → Undo栈大小: 4, Redo栈大小: 0

2️⃣ 当前可以 Undo: True, 可以 Redo: False

3️⃣ 执行两次 Undo:
   undo() → 返回: 状态C - 加了注释
   → Undo栈大小: 3, Redo栈大小: 1
   undo() → 返回: 状态B - 改了颜色
   → Undo栈大小: 2, Redo栈大小: 2

4️⃣ 现在可以 Redo: True

5️⃣ 执行一次 Redo:
   redo() → 返回: 状态C - 加了注释
   → Undo栈大小: 3, Redo栈大小: 1

✅ HistoryStack 演示完成！


In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  Part 3.2: CodeGenerator - 代码生成器                                          ║
# ║  实现"所见即所得，所得即代码"的核心功能                                           ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ============================================================================
# 【核心理念】逆向工程
#
# 传统流程: 代码 → 图表
# 我们的流程: 图表 → 代码（逆向！）
#
# 【实现思路】
# 方法1: 追踪每个用户操作，生成对应代码（复杂，容易出错）
# 方法2: 直接序列化图表状态，生成加载代码（简单，可靠）✅
#
# 我们选择方法2，因为：
# - Plotly 的 Figure 可以完整序列化为 JSON
# - JSON 可以直接用 go.Figure() 重建
# - 无论用户怎么操作，都能正确还原
# ============================================================================

class CodeGenerator:
    """
    代码生成器 - 将当前图表状态转换为可运行的 Python 代码

    ┌─────────────────────────────────────────────────────────────────┐
    │  核心功能：                                                      │
    │  1. generate_code(): 生成基于 JSON 的完整图表代码                 │
    │  2. generate_smart_plot_code(): 生成基于 px 的智能绘图代码        │
    │                                                                  │
    │  【使用场景】                                                     │
    │  - 用户编辑完图表后，点击"导出代码"                                │
    │  - 生成的代码可以直接在 Jupyter/Python 中运行                     │
    │  - 代码可以分享给他人复现图表                                     │
    └─────────────────────────────────────────────────────────────────┘
    """

    def generate_code(self, store: FigureStore) -> str:
        """
        生成完整的图表重建代码

        【原理】
        使用 fig.to_json() 将整个图表序列化为 JSON 字符串
        生成的代码使用 json.loads() 和 go.Figure() 重建

        参数:
            store: FigureStore 实例（包含当前图表）

        返回:
            可直接运行的 Python 代码字符串

        【生成的代码示例】
        ```python
        import json
        import plotly.graph_objects as go

        fig_dict = json.loads('{"data": [...], "layout": {...}}')
        fig = go.Figure(fig_dict)
        fig.show()
        ```
        """
        # 如果没有图表，返回提示信息
        if store.figure is None:
            return "# No figure available yet. Interact with the editor first."

        # 将图表转换为 JSON 字符串
        # 【技术细节】to_json() 返回的是压缩的 JSON，没有换行
        fig_json = store.figure.to_json()

        # 构建代码行
        lines: List[str] = []

        # ━━━━━ 头部注释 ━━━━━
        lines.append("# ============================================================")
        lines.append("# Auto-generated by PyFigureEditor")
        lines.append("# This code recreates the figure exactly as seen in the editor")
        lines.append("# ============================================================")
        lines.append("")

        # ━━━━━ 导入语句 ━━━━━
        lines.append("import json")
        lines.append("import plotly.graph_objects as go")
        lines.append("")

        # ━━━━━ 图表数据 ━━━━━
        # 使用 !r 进行 repr 转义，确保特殊字符正确处理
        lines.append(f"# Complete figure data (serialized)")
        lines.append(f"fig_dict = json.loads({fig_json!r})")
        lines.append("")

        # ━━━━━ 重建图表 ━━━━━
        lines.append("# Reconstruct the figure")
        lines.append("fig = go.Figure(fig_dict)")
        lines.append("")

        # ━━━━━ 显示图表 ━━━━━
        lines.append("# Display the figure")
        lines.append("fig.show()")
        lines.append("")

        # ━━━━━ 提示信息 ━━━━━
        lines.append("# ============================================================")
        lines.append("# Tips: You can now modify the figure programmatically:")
        lines.append("# fig.update_layout(title='My New Title')")
        lines.append("# fig.update_traces(marker_size=10)")
        lines.append("# fig.write_html('my_figure.html')  # Save as HTML")
        lines.append("# fig.write_image('my_figure.png')  # Save as image")
        lines.append("# ============================================================")

        return "\n".join(lines)

    def generate_smart_plot_code(self, df_name: str, plot_type: str, df: pd.DataFrame) -> str:
        """
        生成智能绘图代码 - 自动选择合适的列

        【与 generate_code 的区别】
        - generate_code(): 基于 JSON，100% 精确还原，但代码不可读
        - generate_smart_plot_code(): 基于 px，代码人类可读，但可能需要调整

        【智能选择逻辑】
        1. 检测数值列和分类列
        2. 根据图表类型选择合适的列作为 x, y, color 等
        3. 生成使用 plotly.express 的简洁代码

        参数:
            df_name: DataFrame 变量名 (如 "df", "sales_data")
            plot_type: 图表类型 (如 "scatter", "bar", "pie")
            df: 实际的 DataFrame（用于分析列类型）

        返回:
            人类可读的 Python 代码字符串
        """

        # =====================================================================
        # 步骤1: 分析 DataFrame 的列类型
        # =====================================================================

        # 数值列 - 用于 y 轴、大小、颜色渐变等
        num_cols = df.select_dtypes(include=np.number).columns.tolist()

        # 分类列 - 用于 x 轴标签、颜色分组等
        cat_cols = df.select_dtypes(exclude=np.number).columns.tolist()

        # 所有列
        all_cols = df.columns.tolist()

        # 空数据检查
        if not all_cols:
            return "# Error: Dataset is empty."

        # =====================================================================
        # 步骤2: 辅助函数 - 安全获取列名
        # =====================================================================

        def safe_get(lst, idx):
            """安全获取列表元素，越界返回第一列"""
            return lst[idx] if len(lst) > idx else (all_cols[0] if all_cols else None)

        def safe_get_num(idx):
            """安全获取数值列"""
            return num_cols[idx] if len(num_cols) > idx else (
                num_cols[0] if num_cols else (all_cols[0] if all_cols else None)
            )

        # =====================================================================
        # 步骤3: 根据图表类型选择列
        # =====================================================================
        # 【设计思想】每种图表类型有不同的参数需求

        cols = {}  # 存储选择的列

        # ━━━━━ 散点/折线类 ━━━━━
        if plot_type in ['scatter', 'line', 'area', 'bubble', 'scatter3d', 'line3d', 'ternary']:
            cols['x'] = safe_get_num(0)      # 第一个数值列作为 x
            cols['y'] = safe_get_num(1)      # 第二个数值列作为 y
            cols['z'] = safe_get_num(2)      # 第三个数值列作为 z（3D用）
            cols['size'] = safe_get_num(3)   # 第四个数值列作为大小（气泡图用）
            cols['color'] = safe_get(cat_cols, 0)  # 第一个分类列作为颜色分组

        # ━━━━━ 分类比较类 ━━━━━
        elif plot_type in ['bar', 'pie', 'sunburst', 'treemap', 'funnel']:
            cols['x'] = safe_get(cat_cols, 0) or safe_get(num_cols, 0)  # 优先分类列
            cols['y'] = safe_get_num(0) if cols['x'] != safe_get_num(0) else safe_get_num(1)
            cols['names'] = cols['x']   # 饼图标签
            cols['values'] = cols['y']  # 饼图数值

        # ━━━━━ 分布类 ━━━━━
        elif plot_type in ['hist', 'box', 'violin', 'heatmap']:
            cols['x'] = safe_get_num(0)
            if plot_type == 'hist' and cols['x'] is None:
                cols['x'] = safe_get(all_cols, 0)
            cols['y'] = safe_get_num(1)
            cols['color'] = safe_get(cat_cols, 0)  # 按类别分组

        # ━━━━━ 3D 表面类 ━━━━━
        elif plot_type == 'surface':
            cols['x'], cols['y'], cols['z'] = safe_get_num(0), safe_get_num(1), safe_get_num(2)

        # ━━━━━ 等高线 ━━━━━
        elif plot_type == 'contour':
            cols['x'], cols['y'] = safe_get_num(0), safe_get_num(1)

        # ━━━━━ 极坐标 ━━━━━
        elif plot_type == 'polar':
            cols['x'], cols['y'] = safe_get_num(0), safe_get_num(1)  # r, theta

        # ━━━━━ 瀑布图 ━━━━━
        elif plot_type == 'waterfall':
            cols['x'] = safe_get(cat_cols, 0) or safe_get(all_cols, 0)
            cols['y'] = safe_get_num(0)

        # ━━━━━ 散点矩阵 ━━━━━
        elif plot_type == 'scatmat':
            cols['dimensions'] = num_cols[:5] if len(num_cols) >= 2 else all_cols[:5]
            cols['color'] = safe_get(cat_cols, 0) or safe_get(num_cols, 0)

        # ━━━━━ 平行坐标 ━━━━━
        elif plot_type == 'parcoords':
            cols['dimensions'] = num_cols[:5] if len(num_cols) >= 2 else all_cols[:5]
            cols['color'] = safe_get(num_cols, 0)

        # ━━━━━ K线图（金融） ━━━━━
        elif plot_type == 'candle':
            # 尝试匹配标准金融列名
            lower_cols = [c.lower() for c in all_cols]
            try: cols['x'] = all_cols[lower_cols.index('time')]
            except: cols['x'] = safe_get(all_cols, 0)
            try: cols['open'] = all_cols[lower_cols.index('open')]
            except: cols['open'] = safe_get_num(1)
            try: cols['high'] = all_cols[lower_cols.index('high')]
            except: cols['high'] = safe_get_num(2)
            try: cols['low'] = all_cols[lower_cols.index('low')]
            except: cols['low'] = safe_get_num(3)
            try: cols['close'] = all_cols[lower_cols.index('close')]
            except: cols['close'] = safe_get_num(4)

        # ━━━━━ 地理散点图 ━━━━━
        elif plot_type in ['scatgeo', 'globe']:
            lower_cols = [c.lower() for c in all_cols]
            try: cols['lat'] = all_cols[lower_cols.index('lat')]
            except: cols['lat'] = safe_get_num(0)
            try: cols['lon'] = all_cols[lower_cols.index('lon')]
            except: cols['lon'] = safe_get_num(1)
            cols['color'] = safe_get(cat_cols, 0)

        # ━━━━━ 分级统计图 ━━━━━
        elif plot_type == 'choropleth':
            lower_cols = [c.lower() for c in all_cols]
            try: cols['locations'] = all_cols[lower_cols.index('iso_alpha')]
            except:
                try: cols['locations'] = all_cols[lower_cols.index('country')]
                except: cols['locations'] = safe_get(cat_cols, 0)
            cols['color'] = safe_get_num(0)

        # =====================================================================
        # 步骤4: 生成代码
        # =====================================================================

        cmd = f"# Generate {plot_type} plot from {df_name}\n"
        cmd += "import plotly.express as px\n"
        cmd += "import plotly.graph_objects as go\n\n"

        # 辅助函数：格式化列名
        def fmt_col(key):
            return f"'{cols.get(key)}'" if cols.get(key) else "None"
        def fmt_col_check(key):
            return f"'{cols.get(key)}' if '{cols.get(key)}' != 'None' else None"

        # ━━━━━ 根据类型生成不同的代码 ━━━━━
        if plot_type == 'scatter':
            cmd += f"fig = px.scatter({df_name}, x={fmt_col('x')}, y={fmt_col('y')}, color={fmt_col_check('color')})"
        elif plot_type == 'line':
            cmd += f"fig = px.line({df_name}, x={fmt_col('x')}, y={fmt_col('y')}, color={fmt_col_check('color')})"
        elif plot_type == 'bar':
            cmd += f"fig = px.bar({df_name}, x={fmt_col('x')}, y={fmt_col('y')}, color={fmt_col('x')})"
        elif plot_type == 'area':
            cmd += f"fig = px.area({df_name}, x={fmt_col('x')}, y={fmt_col('y')})"
        elif plot_type == 'bubble':
            cmd += f"fig = px.scatter({df_name}, x={fmt_col('x')}, y={fmt_col('y')}, size={df_name}[{fmt_col('size')}].abs(), color={fmt_col_check('color')})"
        elif plot_type == 'pie':
            cmd += f"fig = px.pie({df_name}, names={fmt_col('names')}, values={fmt_col('values')})"
        elif plot_type == 'sunburst':
            cmd += f"fig = px.sunburst({df_name}, path=[{fmt_col('names')}], values={fmt_col('values')})"
        elif plot_type == 'treemap':
            cmd += f"fig = px.treemap({df_name}, path=[{fmt_col('names')}], values={fmt_col('values')})"
        elif plot_type == 'funnel':
            cmd += f"fig = px.funnel({df_name}, x={fmt_col('values')}, y={fmt_col('names')})"
        elif plot_type == 'hist':
            cmd += f"fig = px.histogram({df_name}, x={fmt_col('x')}, color={fmt_col_check('color')})"
        elif plot_type == 'heatmap':
            cmd += f"fig = px.density_heatmap({df_name}, x={fmt_col('x')}, y={fmt_col('y')})"
        elif plot_type == 'box':
            cmd += f"fig = px.box({df_name}, x={fmt_col_check('color')}, y={fmt_col('y')})"
        elif plot_type == 'violin':
            cmd += f"fig = px.violin({df_name}, x={fmt_col_check('color')}, y={fmt_col('y')})"
        elif plot_type == 'scatter3d':
            cmd += f"fig = px.scatter_3d({df_name}, x={fmt_col('x')}, y={fmt_col('y')}, z={fmt_col('z')}, color={fmt_col_check('color')})"
        elif plot_type == 'line3d':
            cmd += f"fig = px.line_3d({df_name}, x={fmt_col('x')}, y={fmt_col('y')}, z={fmt_col('z')})"
        elif plot_type == 'surface':
            cmd += f"fig = go.Figure(data=[go.Mesh3d(x={df_name}[{fmt_col('x')}], y={df_name}[{fmt_col('y')}], z={df_name}[{fmt_col('z')}], opacity=0.8)])"
        elif plot_type == 'contour':
            cmd += f"fig = px.density_contour({df_name}, x={fmt_col('x')}, y={fmt_col('y')})"
        elif plot_type == 'polar':
            cmd += f"fig = px.scatter_polar({df_name}, r={fmt_col('x')}, theta={fmt_col('y')})"
        elif plot_type == 'ternary':
            cmd += f"fig = px.scatter_ternary({df_name}, a={fmt_col('x')}, b={fmt_col('y')}, c={fmt_col('z')})"
        elif plot_type == 'waterfall':
            cmd += f"fig = go.Figure(go.Waterfall(name='Waterfall', orientation='v', measure=['relative']*len({df_name}), x={df_name}[{fmt_col('x')}], y={df_name}[{fmt_col('y')}], connector={{'mode':'between', 'line':{{'width':4, 'color':'rgb(0, 0, 0)', 'dash':'solid'}}}}))"
        elif plot_type == 'scatmat':
            cmd += f"fig = px.scatter_matrix({df_name}, dimensions={cols.get('dimensions')}, color={fmt_col_check('color')})"
        elif plot_type == 'parcoords':
            cmd += f"fig = px.parallel_coordinates({df_name}, dimensions={cols.get('dimensions')}, color={fmt_col_check('color')})"
        elif plot_type == 'scatgeo':
            cmd += f"fig = px.scatter_geo({df_name}, lat={fmt_col('lat')}, lon={fmt_col('lon')}, color={fmt_col_check('color')})"
        elif plot_type == 'globe':
            cmd += f"fig = px.scatter_geo({df_name}, lat={fmt_col('lat')}, lon={fmt_col('lon')}, color={fmt_col_check('color')}, projection='orthographic')"
        elif plot_type == 'choropleth':
            cmd += f"fig = px.choropleth({df_name}, locations={fmt_col('locations')}, color={fmt_col('color')}, locationmode='ISO-3')"
        elif plot_type == 'candle':
            cmd += f"fig = go.Figure(data=[go.Candlestick(x={df_name}[{fmt_col('x')}], open={df_name}[{fmt_col('open')}], high={df_name}[{fmt_col('high')}], low={df_name}[{fmt_col('low')}], close={df_name}[{fmt_col('close')}])])"

        cmd += "\n\n# Update layout for better appearance\n"
        cmd += "fig.update_layout(template='plotly_white', title='Generated Plot')\n"
        cmd += "\n# Display the figure\n"
        cmd += "fig.show()"

        return cmd


# ============================================================================
# 🔹 演示：CodeGenerator 的使用
# ============================================================================

print("=" * 60)
print("📝 CodeGenerator 演示")
print("=" * 60)

# 创建测试用的 CodeGenerator 和 FigureStore
demo_code_gen = CodeGenerator()
demo_store_for_code = FigureStore()

print("\n1️⃣ generate_code() - 基于 JSON 的完整代码:")
print("-" * 40)
code = demo_code_gen.generate_code(demo_store_for_code)
# 只显示前几行
print("\n".join(code.split("\n")[:10]))
print("... (更多代码省略)")

print("\n2️⃣ generate_smart_plot_code() - 智能绘图代码:")
print("-" * 40)
test_df = pd.DataFrame({
    '日期': ['2024-01', '2024-02', '2024-03'],
    '销售额': [100, 150, 120],
    '区域': ['北京', '上海', '广州']
})
smart_code = demo_code_gen.generate_smart_plot_code("df", "bar", test_df)
print(smart_code)

print("\n✅ CodeGenerator 演示完成！")

📝 CodeGenerator 演示

1️⃣ generate_code() - 基于 JSON 的完整代码:
----------------------------------------
# ============================================================
# Auto-generated by PyFigureEditor
# This code recreates the figure exactly as seen in the editor
# ============================================================

import json
import plotly.graph_objects as go

# Complete figure data (serialized)
fig_dict = json.loads('{"data":[{"line":{"color":"#1f77b4","width":2.5},"marker":{"color":"#1f77b4","size":6.0},"mode":"lines+markers","name":"Demo Signal","visible":true,"x":[0.0,0.05025125628140704,0.10050251256281408,0.15075376884422112,0.20100502512562815,0.2512562814070352,0.30150753768844224,0.35175879396984927,0.4020100502512563,0.45226130653266333,0.5025125628140704,0.5527638190954774,0.6030150753768845,0.6532663316582915,0.7035175879396985,0.7537688442211056,0.8040201005025126,0.8542713567839196,0.9045226130653267,0.9547738693467337,1.0050251256281408,1.0552763819095479,1.105527

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  Part 3.3: 全局单例初始化                                                      ║
# ║  创建应用级别的核心对象实例                                                      ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ============================================================================
# 【设计模式】单例模式 (Singleton Pattern)
#
# 什么是单例？
# - 整个应用只有一个实例
# - 所有地方都访问同一个对象
#
# 为什么需要单例？
# - 状态一致性：所有 Callback 操作同一份数据
# - 资源共享：避免重复创建占用内存
# - 简化访问：不需要传递对象引用
#
# Python 实现方式：
# - 在模块级别创建对象
# - 模块只加载一次，对象自然只有一个
# ============================================================================

# ━━━━━ 核心状态管理器 ━━━━━
figure_store = FigureStore()
# 【作用】管理当前图表、所有数据集、原始数据仓库
# 【使用】所有需要读写图表状态的 Callback 都访问这个实例

# ━━━━━ 历史栈（撤销/重做）━━━━━
history_stack = HistoryStack(max_size=50)
# 【作用】保存图表状态快照，支持 Undo/Redo
# 【参数】max_size=50 表示最多保存50个历史状态

# ━━━━━ 操作日志 ━━━━━
action_log = ActionLog(max_actions=1000)
# 【作用】记录用户操作类型，用于调试和分析
# 【参数】max_actions=1000 表示最多记录1000条日志

# ━━━━━ 代码生成器 ━━━━━
code_generator = CodeGenerator()
# 【作用】将当前图表状态转换为 Python 代码

# ============================================================================
# 辅助函数：创建初始图表
# ============================================================================

def create_initial_figure() -> go.Figure:
    """
    确保 FigureStore 有一个有效的图表

    【调用时机】应用启动时
    【作用】如果没有图表，创建默认演示图表

    返回:
        当前图表对象（go.Figure）
    """
    if figure_store.figure is None:
        figure_store._init_default_figure()
    return cast(go.Figure, figure_store.figure)

# ============================================================================
# 初始化检查与状态输出
# ============================================================================

print("=" * 60)
print("⚙️  核心引擎状态检查")
print("=" * 60)
print(f"   📊 当前主题: {figure_store.current_theme}")
print(f"   📈 数据集数量: {len(figure_store.datasets)}")
print(f"   📁 数据仓库: {list(figure_store.data_repository.keys())}")
print(f"   ⏪ 历史栈容量: {history_stack.max_size}")
print(f"   📝 日志容量: {action_log.max_actions}")

# 将初始状态推入历史栈
# 【重要】确保用户可以撤销第一次修改！
if figure_store.figure:
    history_stack.push(figure_store.get_figure_dict())
    print(f"   ✅ 初始状态已保存到历史栈")

print()
print("🚀 核心引擎初始化完成！")
print("=" * 60)

⚙️  核心引擎状态检查
   📊 当前主题: plotly_white
   📈 数据集数量: 1
   📁 数据仓库: ['demo_signal']
   ⏪ 历史栈容量: 50
   📝 日志容量: 1000
   ✅ 初始状态已保存到历史栈

🚀 核心引擎初始化完成！
   ✅ 初始状态已保存到历史栈

🚀 核心引擎初始化完成！


---
# 🎨 Part 4: UI 组件构建
## 4.1 UI 架构概览

---

### 📐 整体布局设计

```
┌─────────────────────────────────────────────────────────────────────────┐
│  RIBBON (顶部工具栏) - 类似 Microsoft Office 的功能区                     │
│  ┌─────┐ ┌─────┐ ┌─────┐ ┌──────────┐ ┌─────┐                          │
│  │HOME │ │DATA │ │PLOTS│ │ ANNOTATE │ │VIEW │  ← 5个标签页              │
│  └─────┘ └─────┘ └─────┘ └──────────┘ └─────┘                          │
│  [Open][Save][Undo][Redo]  [Import CSV]  [Scatter][Bar]...              │
├─────────────┬───────────────────────────────────┬───────────────────────┤
│  LEFT PANEL │         CENTER CANVAS             │     RIGHT PANEL       │
│  (左侧面板)  │          (中央画布)                │     (右侧面板)         │
│  宽度=3/12  │          宽度=6/12                │      宽度=3/12        │
│             │                                   │                       │
│ ┌─────────┐ │                                   │ ┌───────────────────┐ │
│ │代码编辑器│ │         ┌─────────────┐           │ │ Property Inspector│ │
│ │         │ │         │             │           │ │    (属性检查器)    │ │
│ │ Python  │ │         │   PLOTLY    │           │ ├───────────────────┤ │
│ │ code... │ │         │   GRAPH     │           │ │ 选择元素:          │ │
│ └─────────┘ │         │             │           │ │ [▼ Trace 0     ]  │ │
│             │         │             │           │ │                   │ │
│ ┌─────────┐ │         │             │           │ │ 名称: [Demo     ] │ │
│ │数据表格  │ │         └─────────────┘           │ │ 颜色: [● Blue  ] │ │
│ │         │ │                                   │ │ 大小: [6       ] │ │
│ └─────────┘ │                                   │ │ [Apply] [Delete]  │ │
└─────────────┴───────────────────────────────────┴───────────────────────┘
```

---

### 🧱 Bootstrap 栅格系统

```
【关键概念】Bootstrap 将页面分为 12 列

我们的分配:
├── 左侧面板: 3/12 = 25%
├── 中央画布: 6/12 = 50%  
└── 右侧面板: 3/12 = 25%

代码实现:
dbc.Col([左侧内容], width=3)
dbc.Col([中央内容], width=6)
dbc.Col([右侧内容], width=3)
```

---

### 🎯 组件层次结构

```
app.layout
├── dcc.Store (客户端数据存储)
├── dbc.Container (主容器)
│   ├── ribbon (顶部工具栏)
│   │   ├── dbc.Tabs (标签页)
│   │   └── dbc.CardBody (标签内容)
│   │       ├── HOME: 文件操作、历史
│   │       ├── DATA: 数据管理
│   │       ├── PLOTS: 图表类型按钮
│   │       ├── ANNOTATE: 绘图工具
│   │       └── VIEW: 视图控制
│   │
│   └── dbc.Row (主体行)
│       ├── dbc.Col (左侧, width=3)
│       │   └── workspace_panel
│       │       ├── 代码编辑器
│       │       └── 数据表格
│       │
│       ├── dbc.Col (中央, width=6)
│       │   └── dcc.Graph (Plotly 图表)
│       │
│       └── dbc.Col (右侧, width=3)
│           └── property_inspector
│               ├── 元素选择器
│               └── 属性编辑控件
│
└── dbc.Modal (弹窗对话框)
    └── annotation_modal
```

---

### ❓ 可能的提问与回答

**Q1: 为什么用 `dbc` (dash_bootstrap_components) 而不是原生 `html`？**
> **A:** 原生 Dash 的 `html.Div` 等组件非常基础，需要自己写 CSS 样式。`dbc` 提供了预设计的组件（按钮、卡片、模态框等），开箱即用，大大减少样式工作。

**Q2: `dcc.Store` 是做什么的？**
> **A:** 它是客户端数据存储组件，数据保存在浏览器中（而不是服务器）。用途：
> - 在不同 Callback 之间传递数据
> - 持久化用户会话
> - 减少服务器负载

**Q3: 为什么 `ribbon` 要用 `dbc.Card` 包装？**
> **A:** `dbc.Card` 提供了漂亮的边框、阴影和内边距，让工具栏看起来更专业。不用 Card 的话，内容会直接贴在页面上，没有视觉分隔。

**Q4: 这个布局在手机上能用吗？**
> **A:** 目前设计针对桌面端。Bootstrap 栅格系统支持响应式，但我们的图表编辑器在小屏幕上体验不好。可以通过添加断点规则来优化，但那是额外工作。

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  Part 4.2: UI 组件 - 顶部 Ribbon 工具栏                                        ║
# ║  模仿 Microsoft Office 的功能区设计                                            ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ============================================================================
# 【设计参考】Microsoft Office Ribbon
#
# Ribbon 是微软在 Office 2007 引入的界面设计，特点：
# 1. 标签页分组 (Tabs) - 按功能类别分组
# 2. 按钮组 (Button Groups) - 相关功能放在一起
# 3. 可视化图标 - 用图标+文字让功能一目了然
#
# 我们的 Ribbon 设计：
# ┌─────┬─────┬─────┬──────────┬─────┐
# │HOME │DATA │PLOTS│ ANNOTATE │VIEW │  ← 5个标签页
# └─────┴─────┴─────┴──────────┴─────┘
# ============================================================================

ribbon = dbc.Card([
    # =========================================================================
    # Ribbon 头部：标签页导航
    # =========================================================================
    dbc.CardHeader(
        dbc.Tabs([
            # 每个 Tab 对应一个功能类别
            dbc.Tab(
                label="HOME",           # 标签文字
                tab_id="tab-home",      # 唯一ID (Callback 用)
                label_style={"fontWeight": "bold", "fontSize": "13px"}  # 样式
            ),
            dbc.Tab(label="DATA", tab_id="tab-data",
                    label_style={"fontWeight": "bold", "fontSize": "13px"}),
            dbc.Tab(label="PLOTS", tab_id="tab-plots",
                    label_style={"fontWeight": "bold", "fontSize": "13px"}),
            dbc.Tab(label="ANNOTATE", tab_id="tab-annotate",
                    label_style={"fontWeight": "bold", "fontSize": "13px"}),
            dbc.Tab(label="VIEW", tab_id="tab-view",
                    label_style={"fontWeight": "bold", "fontSize": "13px"}),
        ],
            id="ribbon-tabs",           # 整个 Tabs 组件的 ID
            active_tab="tab-home",      # 默认激活的标签
            className="nav-tabs-bottom" # CSS 类名
        ),
        className="pb-0 border-bottom-0 bg-light pt-1"  # 卡片头部样式
    ),

    # =========================================================================
    # Ribbon 主体：各标签页的内容
    # =========================================================================
    dbc.CardBody([

        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        # HOME 标签页 - 文件操作 & 历史功能
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        html.Div(
            id="ribbon-content-home",   # 用于 Callback 控制显示/隐藏
            children=[
                dbc.Row([
                    # --- 文件操作组 ---
                    dbc.Col([
                        # 分组标题
                        html.Div("File",
                                 className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            # 【上传组件】dcc.Upload - 处理文件上传
                            # 包装了一个按钮，点击后打开文件选择器
                            dcc.Upload(
                                dbc.Button(
                                    [html.Div("📂", className="h4 mb-0"), "Open Session"],
                                    color="light", size="sm",
                                    className="d-flex flex-column align-items-center px-3 border-0"
                                ),
                                id="upload-session",  # 用于接收上传的文件
                                multiple=False,       # 只允许单文件
                                style={"display": "inline-block"}
                            ),
                            # 保存按钮
                            dbc.Button(
                                [html.Div("💾", className="h4 mb-0"), "Save Session"],
                                id="btn-save-session",  # Callback 绑定此 ID
                                color="light", size="sm",
                                className="d-flex flex-column align-items-center px-3 border-0"
                            ),
                        ], className="me-2")
                    ], width="auto", className="border-end pe-2"),

                    # --- 历史操作组 (Undo/Redo) ---
                    dbc.Col([
                        html.Div("History",
                                 className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            # 撤销按钮
                            dbc.Button(
                                "↶ Undo",
                                id="btn-undo",          # Callback 绑定
                                color="link", size="sm",
                                disabled=True,          # 初始禁用（无历史可撤销）
                                className="text-decoration-none text-dark"
                            ),
                            # 重做按钮
                            dbc.Button(
                                "↷ Redo",
                                id="btn-redo",
                                color="link", size="sm",
                                disabled=True,          # 初始禁用
                                className="text-decoration-none text-dark"
                            ),
                        ], vertical=True)  # 垂直排列
                    ], width="auto"),
                ], align="center")
            ],
            style={"display": "block"}  # 默认显示（HOME 是初始标签）
        ),

        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        # DATA 标签页 - 数据管理工作流
        # 工作流: 1.导入 → 2.选择 → 3.检视 → 4.预处理
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        html.Div(
            id="ribbon-content-data",
            children=[
                dbc.Row([
                    # --- 1. 数据源 ---
                    dbc.Col([
                        html.Div("1. Data Source",
                                 className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            # CSV 上传
                            dcc.Upload(
                                dbc.Button(
                                    [html.Div("📂", className="h5 mb-0"), "Import CSV"],
                                    color="light", size="sm",
                                    className="d-flex flex-column align-items-center px-3 border-0"
                                ),
                                id="upload-csv",  # Callback 将处理上传的 CSV
                                multiple=False
                            ),
                            # 加载演示数据
                            dbc.Button(
                                [html.Div("🎲", className="h5 mb-0"), "Load Demo"],
                                id="btn-gen-demo",
                                color="light", size="sm",
                                className="d-flex flex-column align-items-center px-3 border-0"
                            ),
                        ], className="me-2")
                    ], width="auto", className="border-end pe-2"),

                    # --- 2. 活动数据集选择 ---
                    dbc.Col([
                        html.Div("2. Active Dataset",
                                 className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.Row([
                            # 下拉选择器
                            dbc.Col(
                                dcc.Dropdown(
                                    id='dd-dataframe-select',
                                    options=[],           # 初始为空，动态填充
                                    placeholder="Select Data...",
                                    className="small",
                                    style={"width": "180px"}
                                ),
                                className="pe-1"
                            ),
                            # 删除按钮
                            dbc.Col(
                                dbc.Button("🗑️", id="btn-delete-data",
                                          color="danger", outline=True, size="sm"),
                                width="auto", className="ps-0"
                            ),
                        ], align="center", className="g-0"),
                        # 数据信息标签
                        html.Div(
                            id="data-info-label",
                            className="small text-muted mt-1 text-center",
                            style={"fontSize": "9px"},
                            children="No data loaded"
                        )
                    ], width="auto", className="border-end pe-2"),

                    # --- 3. 数据检视 ---
                    dbc.Col([
                        html.Div("3. Inspection",
                                 className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            dbc.Button([html.Div("📋", className="h5 mb-0"), "Raw Table"],
                                      id="btn-view-table", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("📊", className="h5 mb-0"), "Summary"],
                                      id="btn-view-stats", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("ℹ️", className="h5 mb-0"), "Col Types"],
                                      id="btn-view-types", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                        ], className="me-2")
                    ], width="auto", className="border-end pe-2"),

                    # --- 4. 预处理 ---
                    dbc.Col([
                        html.Div("4. Pre-processing",
                                 className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            dbc.Button([html.Div("🧹", className="h5 mb-0"), "Clean NA"],
                                      id="btn-clean-na", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("✂️", className="h5 mb-0"), "Remove Sel"],
                                      id="btn-remove-selected", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🔄", className="h5 mb-0"), "Reset"],
                                      id="btn-reset-data", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                        ])
                    ], width="auto"),
                ], align="center")
            ],
            style={"display": "none"}  # 初始隐藏（不是默认标签）
        ),

        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        # PLOTS 标签页 - 26+ 种图表类型
        # 分组: Basic 2D | Distribution | 3D | Specialized | Maps
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        html.Div(
            id="ribbon-content-plots",
            children=[
                dbc.Row([
                    # --- Basic 2D 图表 ---
                    dbc.Col([
                        html.Div("Basic 2D", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            # 每个按钮都有唯一 ID，格式: btn-plot-{类型}
                            # Callback 会根据 ID 后缀确定要创建的图表类型
                            dbc.Button([html.Div("📈", className="h5 mb-0"), "Scatter"],
                                      id="btn-plot-scatter", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("📉", className="h5 mb-0"), "Line"],
                                      id="btn-plot-line", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("📊", className="h5 mb-0"), "Bar"],
                                      id="btn-plot-bar", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("⛰️", className="h5 mb-0"), "Area"],
                                      id="btn-plot-area", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🫧", className="h5 mb-0"), "Bubble"],
                                      id="btn-plot-bubble", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                        ], className="me-1")
                    ], width="auto", className="border-end pe-1"),

                    # --- 分布图 & 占比图 ---
                    dbc.Col([
                        html.Div("Distribution", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            dbc.Button([html.Div("📶", className="h5 mb-0"), "Hist"],
                                      id="btn-plot-hist", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("📦", className="h5 mb-0"), "Box"],
                                      id="btn-plot-box", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🎻", className="h5 mb-0"), "Violin"],
                                      id="btn-plot-violin", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🔥", className="h5 mb-0"), "Heatmap"],
                                      id="btn-plot-heatmap", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🥧", className="h5 mb-0"), "Pie"],
                                      id="btn-plot-pie", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🍩", className="h5 mb-0"), "Sunburst"],
                                      id="btn-plot-sunburst", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🌳", className="h5 mb-0"), "Treemap"],
                                      id="btn-plot-treemap", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                        ], className="me-1")
                    ], width="auto", className="border-end pe-1"),

                    # --- 3D 图表 ---
                    dbc.Col([
                        html.Div("3D & Contour", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            dbc.Button([html.Div("🧊", className="h5 mb-0"), "Scatter3D"],
                                      id="btn-plot-scatter3d", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🌀", className="h5 mb-0"), "Line3D"],
                                      id="btn-plot-line3d", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🌐", className="h5 mb-0"), "Surface"],
                                      id="btn-plot-surface", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🗺️", className="h5 mb-0"), "Contour"],
                                      id="btn-plot-contour", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                        ], className="me-1")
                    ], width="auto", className="border-end pe-1"),

                    # --- 专业图表 ---
                    dbc.Col([
                        html.Div("Specialized", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            dbc.Button([html.Div("🕸️", className="h5 mb-0"), "Polar"],
                                      id="btn-plot-polar", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🔺", className="h5 mb-0"), "Ternary"],
                                      id="btn-plot-ternary", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🌪️", className="h5 mb-0"), "Funnel"],
                                      id="btn-plot-funnel", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🕯️", className="h5 mb-0"), "Candle"],
                                      id="btn-plot-candle", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🌊", className="h5 mb-0"), "Waterfall"],
                                      id="btn-plot-waterfall", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🔢", className="h5 mb-0"), "ScatMat"],
                                      id="btn-plot-scatmat", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("||", className="h5 mb-0"), "ParCoords"],
                                      id="btn-plot-parcoords", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                        ], className="me-1")
                    ], width="auto", className="border-end pe-1"),

                    # --- 地图图表 ---
                    dbc.Col([
                        html.Div("Maps & Geo", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            dbc.Button([html.Div("🌍", className="h5 mb-0"), "ScatGeo"],
                                      id="btn-plot-scatgeo", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🗺️", className="h5 mb-0"), "Choro"],
                                      id="btn-plot-choropleth", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                            dbc.Button([html.Div("🌐", className="h5 mb-0"), "Globe"],
                                      id="btn-plot-globe", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-2 border-0"),
                        ], className="me-1")
                    ], width="auto", className="border-end pe-1"),
                ], align="center", className="flex-nowrap", style={"overflowX": "auto"})
            ],
            style={"display": "none"}
        ),

        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        # ANNOTATE 标签页 - 绘图和注释工具
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        html.Div(
            id="ribbon-content-annotate",
            children=[
                dbc.Row([
                    # --- 形状绘制 ---
                    dbc.Col([
                        html.Div("Shapes", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            # 这些按钮会触发 Callback 设置 figure 的 dragmode
                            dbc.Button([html.Span("📏"), " Line"], id="btn-draw-line",
                                      outline=True, color="dark", size="sm", className="border-0"),
                            dbc.Button([html.Span("⬜"), " Rect"], id="btn-draw-rect",
                                      outline=True, color="dark", size="sm", className="border-0"),
                            dbc.Button([html.Span("⭕"), " Circle"], id="btn-draw-circle",
                                      outline=True, color="dark", size="sm", className="border-0"),
                            dbc.Button([html.Span("✏️"), " Free"], id="btn-draw-free",
                                      outline=True, color="dark", size="sm", className="border-0"),
                            dbc.Button([html.Span("⬡"), " Poly"], id="btn-draw-poly",
                                      outline=True, color="dark", size="sm", className="border-0"),
                        ])
                    ], width="auto", className="border-end pe-2"),

                    # --- 文字注释 ---
                    dbc.Col([
                        html.Div("Text / Arrow", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.Button([html.Div("📝", className="h4 mb-0"), "Add Annotation"],
                                  id="btn-add-text", color="light", size="sm",
                                  className="d-flex flex-column align-items-center px-3 border-0"),
                    ], width="auto", className="border-end pe-2"),

                    # --- 媒体（图片） ---
                    dbc.Col([
                        html.Div("Media", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dcc.Upload(
                            dbc.Button([html.Div("🖼️", className="h4 mb-0"), "Add Image"],
                                      color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-3 border-0"),
                            id="upload-image",
                            multiple=False
                        ),
                    ], width="auto"),
                ], align="center")
            ],
            style={"display": "none"}
        ),

        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        # VIEW 标签页 - 视图控制
        # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
        html.Div(
            id="ribbon-content-view",
            children=[
                dbc.Row([
                    # --- 导航工具 ---
                    dbc.Col([
                        html.Div("Navigation", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.ButtonGroup([
                            dbc.Button([html.Div("🔍", className="h5 mb-0"), "Zoom"],
                                      id="btn-tool-zoom", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-3 border-0"),
                            dbc.Button([html.Div("✋", className="h5 mb-0"), "Pan"],
                                      id="btn-tool-pan", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-3 border-0"),
                            dbc.Button([html.Div("🏠", className="h5 mb-0"), "Reset"],
                                      id="btn-tool-reset", color="light", size="sm",
                                      className="d-flex flex-column align-items-center px-3 border-0"),
                        ], className="me-2")
                    ], width="auto", className="border-end pe-2"),

                    # --- 面板控制 ---
                    dbc.Col([
                        html.Div("Panels", className="text-muted small fw-bold mb-1 text-center",
                                 style={"fontSize": "10px"}),
                        dbc.Checklist(
                            options=[{"label": "Inspector", "value": "show"}],
                            value=["show"],           # 默认显示检查器
                            id="chk-inspector-toggle",
                            switch=True,              # 使用开关样式
                        )
                    ], width="auto"),
                ], align="center")
            ],
            style={"display": "none"}
        ),

    ])  # End of CardBody

], className="mb-2 shadow-sm")  # 添加阴影效果

# ============================================================================
# 🔹 Ribbon 组件总结
# ============================================================================
print("=" * 60)
print("🎨 Ribbon 工具栏创建完成!")
print("=" * 60)
print(f"   📌 标签页数量: 5 (HOME, DATA, PLOTS, ANNOTATE, VIEW)")
print(f"   📊 图表类型: 26种 (Basic 2D, Distribution, 3D, Specialized, Maps)")
print(f"   🛠️ 总按钮数: ~50+ 个交互控件")

🎨 Ribbon 工具栏创建完成!
   📌 标签页数量: 5 (HOME, DATA, PLOTS, ANNOTATE, VIEW)
   📊 图表类型: 26种 (Basic 2D, Distribution, 3D, Specialized, Maps)
   🛠️ 总按钮数: ~50+ 个交互控件


---
# 🔄 Part 5: Callback 系统详解
## 5.1 什么是 Callback？（核心概念！）

---

### 🎯 Callback 的本质

```
【传统编程 vs 响应式编程】

传统编程 (Polling 轮询):
while True:
    if button.is_clicked():        # 不断检查
        handle_click()
    time.sleep(0.1)                # 浪费 CPU

响应式编程 (Callback 回调):
@app.callback(...)
def handle_click(...):             # 只在事件发生时执行
    ...                            # 高效！
```

---

### 📐 Callback 三要素

```python
@app.callback(
    Output('main-graph', 'figure'),      # 1️⃣ OUTPUT: 要更新什么
    Input('btn-apply', 'n_clicks'),      # 2️⃣ INPUT: 什么触发更新
    State('color-picker', 'value')       # 3️⃣ STATE: 额外读取什么数据
)
def update_graph(n_clicks, color):       # 4️⃣ FUNCTION: 处理逻辑
    ...
    return new_figure                    # 5️⃣ RETURN: 返回值更新 Output
```

---

### 🔑 Input vs State 的区别

| 特性 | Input | State |
|------|-------|-------|
| 值变化时触发回调？ | ✅ 是 | ❌ 否 |
| 可以读取当前值？ | ✅ 是 | ✅ 是 |
| 典型用途 | 按钮点击、下拉选择 | 输入框当前值、设置 |

**示例理解：**
```python
# Input: 用户点击"应用"按钮 → 触发回调
# State: 读取颜色选择器的当前值 → 不触发回调

@app.callback(
    Output('graph', 'figure'),
    Input('btn-apply', 'n_clicks'),      # 点击触发
    State('color-picker', 'value'),       # 只读取值
)
def apply_color(n_clicks, color):
    # n_clicks 变化 → 触发
    # color 变化 → 不触发，但可以读取
    ...
```

---

### 🔄 Callback 执行流程

```
用户操作 (点击按钮)
        ↓
Dash 检测 Input 变化
        ↓
触发对应的 Callback 函数
        ↓
函数执行，返回新值
        ↓
Dash 自动更新 Output 组件
        ↓
用户看到界面变化
```

---

### ❓ 可能的提问与回答

**Q1: 为什么用 `@app.callback` 装饰器而不是普通函数？**
> **A:** 装饰器是 Dash 的"注册机制"。它告诉 Dash："当这些 Input 变化时，调用这个函数，并用返回值更新这些 Output"。没有装饰器，Dash 不知道这个函数的存在。

**Q2: `allow_duplicate=True` 是什么意思？**
> **A:** 默认情况下，一个 Output 只能被一个 Callback 更新。但我们的图表需要被多个 Callback 更新（改颜色、撤销、绘图...），所以要加这个参数允许"多个 Callback 更新同一个 Output"。

**Q3: `prevent_initial_call=True` 的作用？**
> **A:** 阻止应用启动时自动执行一次回调。比如"删除"按钮的回调，启动时执行会删除东西，所以要禁止初始调用。

**Q4: `ctx.triggered_id` 怎么用？**
> **A:** 当一个 Callback 有多个 Input 时，用它判断是哪个触发的：
> ```python
> @app.callback(Output(...), Input('btn-a', 'n_clicks'), Input('btn-b', 'n_clicks'))
> def handle(n_a, n_b):
>     if ctx.triggered_id == 'btn-a':
>         # 处理按钮 A
>     elif ctx.triggered_id == 'btn-b':
>         # 处理按钮 B
> ```

**Q5: `dash.no_update` 是什么？**
> **A:** 当你有多个 Output 但只想更新其中一部分时使用。返回 `dash.no_update` 表示"这个 Output 保持原样不变"。

**Q6: 回调函数可以不返回值吗？**
> **A:** 不行！必须返回与 Output 数量匹配的值。如果确实不想更新，用 `raise PreventUpdate` 或返回 `dash.no_update`。

In [ ]:
# ====================================================================
# 8. UI Components: Workspace & Inspector
# ====================================================================

# --- Workspace Panel (Left) ---
workspace_panel = dbc.Card([
    dbc.CardHeader(
        dbc.Tabs([
            dbc.Tab(label="Command Window", tab_id="tab-cmd", label_style={"fontSize": "12px"}),
            dbc.Tab(label="Data View", tab_id="tab-dataview", label_style={"fontSize": "12px"}),
        ], id="workspace-tabs", active_tab="tab-cmd", className="nav-tabs-bottom"),
        className="py-1 px-2 bg-light"
    ),
    dbc.CardBody([
        # Command Window Content
        html.Div(id="workspace-content-cmd", children=[
            dcc.Textarea(
                id='code-editor',
                value="# Python Command Window\n# Select a dataset and click a plot button...",
                style={'width': '100%', 'height': '300px', 'fontFamily': 'Consolas, monospace', 'fontSize': '13px', 'backgroundColor': '#f8f9fa', 'border': '1px solid #dee2e6'},
                className="mb-2"
            ),
            dbc.Button("▶ Run Code", id="btn-run-custom-code", color="success", size="sm", className="w-100 mb-2"),
            html.Div("Console Output:", className="small fw-bold text-muted"),
            html.Div(
                id="console-output",
                style={'width': '100%', 'height': '150px', 'fontFamily': 'Consolas, monospace', 'fontSize': '12px', 'backgroundColor': '#212529', 'color': '#00ff00', 'padding': '5px', 'overflowY': 'auto'},
                children=">>> Ready."
            )
        ], style={"display": "block"}),

        # Data View Content
        html.Div(id="workspace-content-dataview", children=[
            html.Div(id="data-table-container", children="No data loaded.")
        ], style={"display": "none"}),
    ], className="p-2")
], className="h-100 border-0")

# --- Property Inspector (Right) ---
property_inspector = dbc.Card([
    dbc.CardHeader("Property Inspector", className="bg-light py-1 px-2 small fw-bold"),
    dbc.CardBody([
        # 1. Element Selector
        html.Div([
            html.Label("Select Element:", className="small fw-bold"),
            dbc.Row([
                dbc.Col(dcc.Dropdown(
                    id='dd-element-select',
                    placeholder="Select element...",
                    className="small mb-2",
                    clearable=False,
                    style={"fontSize": "12px"}
                ), width=9, className="pe-1"),
                dbc.Col(dbc.Button("🔦", id="btn-highlight", color="warning", size="sm", outline=True, title="Highlight Selected"), width=3, className="ps-0"),
            ], className="g-0"),
        ], className="mb-3 border-bottom pb-2"),

        # 2. Dynamic Properties Container
        html.Div(id="inspector-controls", children=[
            html.Div("Select an element above to edit its properties.", className="text-muted small text-center mt-5")
        ])

    ], style={"overflowY": "auto", "padding": "10px"})
], className="h-100 border-start rounded-0")

# --- Annotation Modal ---
annotation_modal = dbc.Modal([
    dbc.ModalHeader("Add Annotation / Arrow"),
    dbc.ModalBody([
        dbc.Label("Text Content:"),
        dbc.Input(id="annot-text", placeholder="Enter text...", className="mb-3"),
        dbc.Row([
            dbc.Col([dbc.Label("X Coordinate:"), dbc.Input(id="annot-x", type="number", placeholder="Auto (Center)")], width=6),
            dbc.Col([dbc.Label("Y Coordinate:"), dbc.Input(id="annot-y", type="number", placeholder="Auto (Center)")], width=6),
        ], className="mb-3"),
        dbc.Checklist(
            options=[{"label": "Show Arrow", "value": "arrow"}],
            value=["arrow"],
            id="annot-arrow",
            switch=True
        )
    ]),
    dbc.ModalFooter(
        dbc.Button("Add Annotation", id="btn-confirm-annot", color="primary", n_clicks=0)
    )
], id="modal-annotation", is_open=False)

# --- Main Layout ---
app.layout = html.Div([
    dcc.Store(id='figure-store-client', data=figure_store.get_figure_dict()),
    dcc.Store(id='active-dataframe-name', data="demo_signal"),
    dcc.Store(id='trigger-run-signal', data=0), # Signal to auto-run code
    dcc.Store(id='data-update-signal', data=0), # Signal to refresh data view
    dcc.Download(id="download-component"),

    # Top Ribbon
    ribbon,

    # Main Workspace Area
    dbc.Container([
        dbc.Row([
            # Left: Workspace / Command Window
            dbc.Col(workspace_panel, width=3, className="pe-0 border-end", style={"height": "calc(100vh - 100px)"}),

            # Center: Canvas
            dbc.Col([
                dcc.Graph(
                    id='main-graph',
                    figure=create_initial_figure(),
                    config={
                        'editable': True,
                        'scrollZoom': True,
                        'modeBarButtonsToAdd': ['drawline', 'drawopenpath', 'drawclosedpath', 'drawcircle', 'drawrect', 'eraseshape']
                    },
                    style={'height': '100%', 'width': '100%'}
                )
            ], width=6, className="p-0", style={"height": "calc(100vh - 100px)"}),

            # Right: Inspector
            dbc.Col(property_inspector, width=3, className="ps-0", id="col-inspector", style={"height": "calc(100vh - 100px)"}),
        ], className="g-0 h-100")
    ], fluid=True, className="px-0 h-100"),

    annotation_modal,

], style={"height": "100vh", "overflow": "hidden", "backgroundColor": "#f4f6f8"})

---
## 5.2 Callback 实战分析：数据管理

---

### 📊 数据管理 Callback 结构图

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     manage_data() Callback                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  INPUTS (触发器):                                                        │
│  ├── upload-csv.contents    → CSV 文件上传                               │
│  ├── btn-gen-demo.n_clicks  → "Load Demo" 按钮点击                       │
│  ├── btn-delete-data.n_clicks → "删除" 按钮点击                          │
│  └── btn-clean-na.n_clicks  → "Clean NA" 按钮点击                        │
│                                                                         │
│  STATES (读取但不触发):                                                   │
│  ├── upload-csv.filename    → 上传文件的文件名                            │
│  ├── dd-dataframe-select.value → 当前选中的数据集                         │
│  └── data-update-signal.data → 数据更新信号                               │
│                                                                         │
│  OUTPUTS (更新目标):                                                      │
│  ├── dd-dataframe-select.options → 下拉选项列表                           │
│  ├── dd-dataframe-select.value   → 当前选中值                             │
│  ├── data-info-label.children    → 数据信息文本                           │
│  ├── data-update-signal.data     → 更新信号（通知其他回调）                │
│  └── main-graph.figure           → 图表（删除数据时需要同步）              │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

### 🔑 关键技术点

**1. `ctx.triggered_id` - 判断触发源**
```python
ctx_id = ctx.triggered_id
if ctx_id == "upload-csv":
    # 处理上传
elif ctx_id == "btn-gen-demo":
    # 生成演示数据
elif ctx_id == "btn-delete-data":
    # 删除数据
```

**2. Base64 文件解码**
```python
# 浏览器上传的文件是 Base64 编码的
_content_type, content_string = upload_content.split(',')
decoded = base64.b64decode(content_string)  # 解码为二进制
df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))  # 转为 DataFrame
```

**3. 信号机制 (`data-update-signal`)**
```python
# 问题：多个 Callback 需要知道"数据变了"
# 解决：用一个隐藏的 dcc.Store 作为信号
current_signal += 1  # 信号值 +1 = "数据有更新"
# 其他 Callback 监听这个信号，就知道要刷新了
```

---

### ❓ 可能的提问与回答

**Q1: 为什么一个 Callback 有这么多 Input？**
> **A:** 这些 Input 都会导致数据变化（上传、生成、删除、清理），逻辑上属于同一个"数据管理"功能。用一个 Callback 统一处理可以避免冲突。

**Q2: 生成的演示数据为什么这么复杂？**
> **A:** 为了支持所有 26 种图表类型！需要：
> - 数值列 (x_val, y_val, z_val) - 散点图、3D
> - 分类列 (category) - 饼图、柱状图
> - 时间列 (time) - 时序图
> - OHLC 列 (open, high, low, close) - K线图
> - 地理列 (lat, lon, iso_alpha) - 地图

**Q3: `dash.no_update` 什么时候用？**
> **A:** 当有多个 Output 但只需要更新部分时。比如上传数据时，不需要更新图表（fig_update = dash.no_update），只有删除数据时才需要同步更新图表。

In [ ]:
# ====================================================================
# 9. Callbacks: UI Interaction
# ====================================================================

# --- Ribbon & Workspace Tab Switching ---
@app.callback(
    Output("ribbon-content-home", "style"),
    Output("ribbon-content-data", "style"),
    Output("ribbon-content-plots", "style"),
    Output("ribbon-content-annotate", "style"),
    Output("ribbon-content-view", "style"),
    Input("ribbon-tabs", "active_tab"),
)
def toggle_ribbon(active_tab):
    show = {"display": "block"}
    hide = {"display": "none"}
    return (
        show if active_tab == "tab-home" else hide,
        show if active_tab == "tab-data" else hide,
        show if active_tab == "tab-plots" else hide,
        show if active_tab == "tab-annotate" else hide,
        show if active_tab == "tab-view" else hide,
    )

@app.callback(
    Output("workspace-content-cmd", "style"),
    Output("workspace-content-dataview", "style"),
    Input("workspace-tabs", "active_tab"),
)
def toggle_workspace(active_tab):
    show = {"display": "block"}
    hide = {"display": "none"}
    return (show if active_tab == "tab-cmd" else hide, show if active_tab == "tab-dataview" else hide)

@app.callback(
    Output("col-inspector", "style"),
    Input("chk-inspector-toggle", "value"),
)
def toggle_inspector(value):
    if not value:
        return {"display": "none"}
    return {"display": "block", "height": "calc(100vh - 100px)"}

In [ ]:
# ====================================================================
# 10a. Callbacks: Data Management
# ====================================================================

@app.callback(
    Output("dd-dataframe-select", "options"), # Added missing output
    Output("dd-dataframe-select", "value"),
    Output("data-info-label", "children"),
    Output("data-update-signal", "data"), # Primary owner
    Output("main-graph", "figure", allow_duplicate=True), # Sync plot with data changes
    Input("upload-csv", "contents"),
    Input("btn-gen-demo", "n_clicks"),
    Input("btn-delete-data", "n_clicks"),
    Input("btn-clean-na", "n_clicks"),
    State("upload-csv", "filename"),
    State("dd-dataframe-select", "value"),
    State("data-update-signal", "data"),
    prevent_initial_call=True
)
def manage_data(upload_content, _n_demo, _n_delete, _n_clean, filename, current_selection, current_signal):
    ctx_id = ctx.triggered_id
    current_signal = current_signal or 0
    fig_update = dash.no_update

    # Handle CSV Upload
    if ctx_id == "upload-csv" and upload_content:
        _content_type, content_string = upload_content.split(',')
        decoded = base64.b64decode(content_string)
        try:
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
            name = filename.split('.')[0]
            figure_store.add_dataframe(name, df)
            current_selection = name.replace(" ", "_").replace("-", "_").replace(".", "_") # Auto-select new data
            current_signal += 1
        except Exception as e:
            print(f"Error parsing CSV: {e}")

    # Handle Demo Data Generation
    if ctx_id == "btn-gen-demo":
        # Generate a richer dataset suitable for ALL plot types
        n_points = 200
        t = np.linspace(0, 10, n_points)

        # Categories for Pie/Bar
        categories = np.random.choice(['Category A', 'Category B', 'Category C', 'Category D'], n_points)

        # OHLC for Candlestick
        price = 100 + np.cumsum(np.random.randn(n_points))
        high = price + np.random.rand(n_points) * 5
        low = price - np.random.rand(n_points) * 5
        open_p = price + np.random.randn(n_points)
        close_p = price + np.random.randn(n_points)

        # Geo Data for Maps
        lat = np.random.uniform(-50, 70, n_points)
        lon = np.random.uniform(-120, 140, n_points)
        countries = np.random.choice(['USA', 'CAN', 'GBR', 'FRA', 'DEU', 'CHN', 'JPN', 'BRA', 'IND', 'AUS'], n_points)
        iso_map = {'USA': 'USA', 'CAN': 'CAN', 'GBR': 'GBR', 'FRA': 'FRA', 'DEU': 'DEU', 'CHN': 'CHN', 'JPN': 'JPN', 'BRA': 'BRA', 'IND': 'IND', 'AUS': 'AUS'}
        iso_codes = [iso_map[c] for c in countries]

        df = pd.DataFrame({
            "time": t,
            "signal": np.sin(t) * 10 + np.random.normal(0, 1, n_points),
            "noise": np.random.randn(n_points),
            "category": categories,
            "x_val": np.random.randn(n_points) * 10,
            "y_val": np.random.randn(n_points) * 10,
            "z_val": np.random.randn(n_points) * 10,  # Explicit Z column for 3D
            "size_val": np.random.randint(5, 20, n_points), # For Bubble
            "open": open_p, "high": high, "low": low, "close": close_p, # For Candle
            "lat": lat, "lon": lon, "country": countries, "iso_alpha": iso_codes # For Maps
        })
        name = f"demo_{uuid.uuid4().hex[:4]}"
        figure_store.add_dataframe(name, df)
        current_selection = name
        current_signal += 1

    # Handle Deletion
    if ctx_id == "btn-delete-data" and current_selection:
        if current_selection in figure_store.data_repository:
            del figure_store.data_repository[current_selection]

            # Also remove from datasets (traces) to keep plot in sync
            keys_to_remove = [k for k, d in figure_store.datasets.items() if d.name == current_selection]
            for k in keys_to_remove:
                del figure_store.datasets[k]
                if k in figure_store.dataset_order:
                    figure_store.dataset_order.remove(k)

            if keys_to_remove:
                figure_store.rebuild_figure_from_datasets()
                fig_update = figure_store.get_figure_dict()

            current_selection = None
            current_signal += 1

    # Handle Cleaning
    if ctx_id == "btn-clean-na" and current_selection:
        df = figure_store.get_dataframe(current_selection)
        if df is not None:
            # Smart Cleaning Logic
            for col in df.columns:
                if df[col].dtype == 'object':
                    numeric_col = pd.to_numeric(df[col], errors='coerce')
                    num_valid = numeric_col.count()
                    total_valid = df[col].count()
                    if total_valid > 0 and (num_valid / total_valid) > 0.5:
                        df[col] = numeric_col

            df = df.dropna()
            figure_store.add_dataframe(current_selection, df)

            # Update Datasets & Figure to reflect cleaned data
            updated_datasets = False
            for key, dataset in figure_store.datasets.items():
                if dataset.name == current_selection:
                    dataset.df = df
                    updated_datasets = True

            if updated_datasets:
                figure_store.rebuild_figure_from_datasets()
                fig_update = figure_store.get_figure_dict()

            current_signal += 1

    # Update Options
    options = [{"label": k, "value": k} for k in figure_store.data_repository.keys()]

    # Fallback selection
    if current_selection not in figure_store.data_repository:
        current_selection = options[-1]["value"] if options else None

    # Info Label
    info_text = "No data loaded"
    if current_selection:
        df = figure_store.get_dataframe(current_selection)
        if df is not None:
            info_text = f"{len(df)} rows × {len(df.columns)} cols"

    return options, current_selection, info_text, current_signal, fig_update

@app.callback(
    Output("data-table-container", "children"),
    Output("workspace-tabs", "active_tab"),
    Input("dd-dataframe-select", "value"),
    Input("btn-view-table", "n_clicks"),
    Input("btn-view-stats", "n_clicks"),
    Input("btn-view-types", "n_clicks"),
    Input("data-update-signal", "data"), # Listen to signal
    State("workspace-tabs", "active_tab"),
)
def update_data_view(df_name, _n_table, _n_stats, _n_types, _signal, active_tab):
    ctx_id = ctx.triggered_id

    # Auto-switch to Data View tab if buttons clicked OR if new data selected
    if ctx_id in ["btn-view-table", "btn-view-stats", "btn-view-types", "dd-dataframe-select"]:
        active_tab = "tab-dataview"

    if not df_name:
        return "No data selected.", active_tab

    df = figure_store.get_dataframe(df_name)
    if df is None:
        return "Data not found.", active_tab

    # Determine content type
    if ctx_id == "btn-view-stats":
        # Show Statistics
        stats_df = df.describe().reset_index()
        content = dash_table.DataTable(
            data=cast(List[Dict[str, Any]], stats_df.to_dict('records')),
            columns=[{"name": i, "id": i} for i in stats_df.columns],
            style_table={'overflowX': 'auto'},
            style_cell={'fontSize': '12px', 'fontFamily': 'Arial', 'textAlign': 'left'},
            style_header={'fontWeight': 'bold', 'backgroundColor': '#f8f9fa'}
        )
    elif ctx_id == "btn-view-types":
        # Show Column Types
        types_df = pd.DataFrame(df.dtypes, columns=['Dtype']).reset_index().rename(columns={'index': 'Column'})
        types_df['Dtype'] = types_df['Dtype'].astype(str)
        content = dash_table.DataTable(
            data=cast(List[Dict[str, Any]], types_df.to_dict('records')),
            columns=[{"name": i, "id": i} for i in types_df.columns],
            style_cell={'fontSize': '12px', 'fontFamily': 'Arial', 'textAlign': 'left'},
            style_header={'fontWeight': 'bold', 'backgroundColor': '#f8f9fa'}
        )
    else:
        # Default: Show Data Table (First 100 rows)
        # ENABLE EDITING HERE
        content = dash_table.DataTable(
            id='interactive-data-table',
            data=cast(List[Dict[str, Any]], df.head(100).to_dict('records')),
            columns=[{"name": i, "id": i, "editable": True} for i in df.columns],
            editable=True,
            row_deletable=True,
            style_table={'overflowX': 'auto'},
            style_cell={'fontSize': '12px', 'fontFamily': 'Arial', 'textAlign': 'left'},
            page_size=20,
            style_header={'fontWeight': 'bold', 'backgroundColor': '#e9ecef'}
        )

    return content, active_tab

# New Callback: Sync Edited Table Data back to Repository
@app.callback(
    Output("console-output", "children", allow_duplicate=True),
    Output("main-graph", "figure", allow_duplicate=True), # Add figure output
    Input("interactive-data-table", "data"),
    State("dd-dataframe-select", "value"),
    State("console-output", "children"),
    prevent_initial_call=True
)
def sync_data_from_table(rows, df_name, current_console):
    if not rows or not df_name:
        raise PreventUpdate

    # Convert back to DataFrame
    try:
        # 1. Get original DF to preserve types if possible
        original_df = figure_store.get_dataframe(df_name)
        new_df = pd.DataFrame(rows)

        # Attempt to restore types
        if original_df is not None:
            for col in new_df.columns:
                if col in original_df.columns:
                    try:
                        # Try to cast to original type
                        if pd.api.types.is_numeric_dtype(original_df[col]):
                            new_df[col] = pd.to_numeric(new_df[col], errors='coerce')
                        elif pd.api.types.is_datetime64_any_dtype(original_df[col]):
                            new_df[col] = pd.to_datetime(new_df[col], errors='coerce')
                        # Add more type checks as needed
                    except:
                        pass # Keep as is if cast fails

        # 2. Update Repository
        figure_store.add_dataframe(df_name, new_df)

        # 3. Update Datasets (Traces) that use this dataframe
        # This ensures that if we re-plot or interact, the traces are in sync
        updated_count = 0
        for key, dataset in figure_store.datasets.items():
            if dataset.name == df_name:
                dataset.df = new_df
                updated_count += 1

        msg = f">>> Data '{df_name}' updated from table ({len(new_df)} rows)."
        fig_update = dash.no_update

        if updated_count > 0:
            figure_store.rebuild_figure_from_datasets()
            fig_update = figure_store.get_figure_dict()
            msg += f" Synced {updated_count} active traces."

        return f"{current_console}\n{msg}", fig_update
    except Exception as e:
        return f"{current_console}\n>>> Error updating data: {e}", dash.no_update

In [ ]:
# ====================================================================
# 10b. Callbacks: Data Interaction (Selection & Removal)
# ====================================================================

@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Output("main-graph", "selectedData"), # Explicitly clear selection
    Output("console-output", "children", allow_duplicate=True),
    Output("data-update-signal", "data", allow_duplicate=True),
    Input("btn-remove-selected", "n_clicks"),
    State("main-graph", "selectedData"),
    State("main-graph", "figure"),
    State("dd-dataframe-select", "value"),
    State("data-update-signal", "data"),
    prevent_initial_call=True
)
def remove_selected_points(n_clicks, selected_data, fig_dict, df_name, current_signal):
    if not n_clicks:
        raise PreventUpdate

    if not selected_data:
        return dash.no_update, dash.no_update, ">>> No points selected to remove. (selectedData is None)", dash.no_update

    points = selected_data.get("points", [])
    if not points:
        keys = list(selected_data.keys())
        return dash.no_update, dash.no_update, f">>> Selection is empty. Keys found: {keys}", dash.no_update

    # Debug info
    first_pt = points[0]
    print(f"DEBUG: Removing {len(points)} points. Sample: {first_pt}")

    msg = ""
    current_signal = current_signal or 0
    data_changed = False

    # 1. Try to update the underlying Data Repository (The "Real" Data)
    if df_name and df_name in figure_store.data_repository:
        try:
            df = figure_store.data_repository[df_name]
            indices_to_drop = set()
            for pt in points:
                p_idx = pt.get("pointIndex")
                if p_idx is not None and 0 <= p_idx < len(df):
                    indices_to_drop.add(df.index[p_idx])

            if indices_to_drop:
                new_df = df.drop(list(indices_to_drop))
                figure_store.add_dataframe(df_name, new_df)

                # Sync datasets (traces) to ensure consistency
                updated_count = 0
                for key, dataset in figure_store.datasets.items():
                    if dataset.name == df_name:
                        dataset.df = new_df
                        updated_count += 1

                msg += f">>> Removed {len(indices_to_drop)} rows from dataset '{df_name}'. (Synced {updated_count} traces)\n"
                data_changed = True
            else:
                msg += f">>> No matching rows found in '{df_name}' (Index mismatch?).\n"
        except Exception as e:
            msg += f">>> Error updating dataset: {e}\n"
    else:
        msg += ">>> No active dataset selected. Only updating plot.\n"

    # 2. Update the Visual Figure (The "Image")
    try:
        # Group points by curveNumber
        points_by_curve = defaultdict(list)
        for pt in points:
            points_by_curve[pt['curveNumber']].append(pt['pointIndex'])

        # We edit the fig_dict directly to preserve exactly what is on screen
        fig = go.Figure(fig_dict)
        any_visual_change = False

        for curve_idx, p_indices in points_by_curve.items():
            if curve_idx < len(fig.data):
                trace = fig.data[curve_idx]
                # Only handle traces with x/y arrays
                if hasattr(trace, 'x') and trace.x is not None:
                    x_list = list(trace.x)
                    y_list = list(trace.y) if hasattr(trace, 'y') and trace.y is not None else []

                    # Create a mask for keeping points
                    p_indices_set = set(p_indices)
                    mask = [i not in p_indices_set for i in range(len(x_list))]

                    # Apply mask
                    new_x = [x for i, x in enumerate(x_list) if mask[i]]
                    updates = {'x': new_x}

                    if y_list:
                        new_y = [y for i, y in enumerate(y_list) if mask[i]]
                        updates['y'] = new_y

                    # Also handle marker colors/sizes if they are arrays
                    # Use getattr to safely access marker, and check for color
                    marker = getattr(trace, 'marker', None)
                    if marker:
                        color = getattr(marker, 'color', None)
                        if isinstance(color, (list, tuple, np.ndarray)) and len(color) == len(x_list):
                            new_color = [c for i, c in enumerate(color) if mask[i]]
                            # Use nested update for marker
                            updates['marker'] = dict(color=new_color)

                    trace.update(updates)
                    any_visual_change = True

        new_signal = (current_signal + 1) if data_changed else dash.no_update

        # Force clear selections in layout just in case
        fig.update_layout(selections=[])

        if any_visual_change:
            figure_store.update_figure(fig)
            # Return None for selectedData to clear it on the client side
            return fig, None, msg + " (Visual update applied).", new_signal
        else:
            return dash.no_update, dash.no_update, msg + " (No visual changes possible).", new_signal

    except Exception as e:
        return dash.no_update, dash.no_update, msg + f" (Visual update failed: {e})", dash.no_update

---
## 5.3 Callback 实战分析：代码生成与执行系统

---

### 🎯 设计目标

这是整个应用最独特的创新点：**动态代码生成 + 安全执行**。用户点击图表类型按钮后，系统自动分析数据结构，生成合适的 Plotly 代码，然后执行。

---

### 📐 双 Callback 协作架构

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    两个 Callback 的协作流程                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  第一阶段: generate_and_trigger_plot()                                   │
│  ┌──────────────────────────────────────────────────────────────────┐   │
│  │  触发: 用户点击 26 种图表按钮中的任意一个                            │   │
│  │                      ↓                                            │   │
│  │  分析: ctx.triggered_id → 提取图表类型 (scatter/bar/pie...)        │   │
│  │                      ↓                                            │   │
│  │  生成: CodeGenerator.generate_smart_plot_code()                   │   │
│  │                      ↓                                            │   │
│  │  输出: code-editor.value + trigger-run-signal + 1                 │   │
│  └──────────────────────────────────────────────────────────────────┘   │
│                           │                                             │
│                           │ (Signal触发)                                │
│                           ↓                                             │
│  第二阶段: execute_code()                                                │
│  ┌──────────────────────────────────────────────────────────────────┐   │
│  │  触发: trigger-run-signal 变化 或 用户点击 "Run Code" 按钮          │   │
│  │                      ↓                                            │   │
│  │  构建: 创建安全的执行作用域 (local_scope)                            │   │
│  │        - 注入: pd, px, go, np, figure_store                       │   │
│  │        - 注入: 所有数据集 (demo_signal, test_data, etc.)           │   │
│  │                      ↓                                            │   │
│  │  执行: exec(code, {}, local_scope)                                │   │
│  │                      ↓                                            │   │
│  │  提取: local_scope["fig"] → 更新画布                               │   │
│  └──────────────────────────────────────────────────────────────────┘   │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

### 🔑 关键技术解析

#### 1. Signal 模式（信号触发）

```python
# 问题: 如何让"代码生成"完成后自动触发"代码执行"?
# 方案: 使用 dcc.Store 作为信号传递

# Callback 1 输出:
Output("trigger-run-signal", "data")  # ← 信号值 +1

# Callback 2 输入:
Input("trigger-run-signal", "data")   # ← 监听信号变化

# 这样 Callback 1 完成后，Callback 2 自动触发!
```

#### 2. 安全代码执行（exec 沙箱）

```python
# 为什么用 exec()？
# → 用户可以在 Code Editor 中写任意 Python 代码

# 安全措施:
local_scope = {
    "pd": pd,
    "px": px,
    "go": go,
    "np": np,
    "figure_store": figure_store
}
exec(code, {}, local_scope)  # globals={} 限制全局访问

# 注意: exec 不是完全安全的! 生产环境需要更严格的沙箱
```

#### 3. 26 个 Input 的优化写法

```python
@app.callback(
    Output("code-editor", "value"),
    Output("trigger-run-signal", "data"),
    Input("btn-plot-scatter", "n_clicks"),  # 26 个按钮
    Input("btn-plot-line", "n_clicks"),
    # ... 更多按钮
    prevent_initial_call=True  # ← 必须! 否则页面加载时就会触发
)
def generate_and_trigger_plot(_n_sc, _n_ln, ...):  # 参数用 _ 前缀表示"不使用"
    ctx_id = ctx.triggered_id  # ← 只需要知道"谁触发的"
    plot_type = ctx_id.replace("btn-plot-", "")  # 提取图表类型
```

---

### ❓ 可能的提问与回答

**Q1: 为什么把代码生成和执行分成两个 Callback？**
> **A:** 分离关注点!
> - generate_and_trigger_plot() → 只负责分析数据、生成代码
> - execute_code() → 只负责执行代码、更新图表
>
> 这样用户还可以**手动编辑生成的代码**后再执行，增加灵活性。

**Q2: exec() 不安全，为什么还用？**
> **A:** 教学演示用途，可接受的风险。生产环境应该用：
> - RestrictedPython (限制语法)
> - Docker 容器隔离
> - AST 白名单检查

**Q3: 为什么参数用下划线前缀如 `_n_sc`？**
> **A:** Python 惯例，表示"这个参数我接收了但不使用"。所有 26 个按钮的 n_clicks 我都不关心具体值，只用 `ctx.triggered_id` 判断来源。

**Q4: local_scope 里为什么要注入所有 DataFrame？**
> **A:** 让用户代码可以直接使用变量名！
> ```python
> # 用户在 Code Editor 中可以写:
> fig = px.scatter(demo_signal, x="date", y="value")
> # 而不需要:
> fig = px.scatter(figure_store.get_dataframe("demo_signal"), ...)
> ```

---

In [ ]:
# ====================================================================
# 11. Callbacks: Code Generation & Execution
# ====================================================================

@app.callback(
    Output("code-editor", "value"),
    Output("trigger-run-signal", "data"), # Signal to auto-run
    Input("btn-plot-scatter", "n_clicks"),
    Input("btn-plot-line", "n_clicks"),
    Input("btn-plot-bar", "n_clicks"),
    Input("btn-plot-area", "n_clicks"),
    Input("btn-plot-bubble", "n_clicks"),
    Input("btn-plot-pie", "n_clicks"),
    Input("btn-plot-sunburst", "n_clicks"),
    Input("btn-plot-treemap", "n_clicks"),
    Input("btn-plot-heatmap", "n_clicks"),
    Input("btn-plot-scatter3d", "n_clicks"),
    Input("btn-plot-line3d", "n_clicks"),
    Input("btn-plot-surface", "n_clicks"),
    Input("btn-plot-contour", "n_clicks"),
    Input("btn-plot-hist", "n_clicks"),
    Input("btn-plot-box", "n_clicks"),
    Input("btn-plot-violin", "n_clicks"),
    Input("btn-plot-polar", "n_clicks"),
    Input("btn-plot-ternary", "n_clicks"),
    Input("btn-plot-funnel", "n_clicks"),
    Input("btn-plot-candle", "n_clicks"),
    Input("btn-plot-waterfall", "n_clicks"),
    Input("btn-plot-scatmat", "n_clicks"),
    Input("btn-plot-parcoords", "n_clicks"),
    Input("btn-plot-scatgeo", "n_clicks"),
    Input("btn-plot-choropleth", "n_clicks"),
    Input("btn-plot-globe", "n_clicks"),
    State("dd-dataframe-select", "value"),
    State("trigger-run-signal", "data"),
    prevent_initial_call=True
)
def generate_and_trigger_plot(_n_sc, _n_ln, _n_bar, _n_area, _n_bub, _n_pie, _n_sun, _n_tree, _n_heat, _n_3d, _n_line3d, _n_surf, _n_cont, _n_hist, _n_box, _n_violin, _n_pol, _n_ter, _n_fun, _n_can, _n_wat, _n_smat, _n_par, _n_geo, _n_choro, _n_globe, df_name, current_signal):
    if not ctx.triggered_id:
        raise PreventUpdate

    if not df_name:
        return "# Please select a dataset first.", dash.no_update

    ctx_id = ctx.triggered_id
    plot_type = ctx_id.replace("btn-plot-", "")

    df = figure_store.get_dataframe(df_name)
    if df is None:
        return "# Error: Dataset not found.", dash.no_update

    # Use the enhanced CodeGenerator logic
    cmd = code_generator.generate_smart_plot_code(df_name, plot_type, df)

    # Increment signal to trigger execution
    new_signal = (current_signal or 0) + 1
    return cmd, new_signal

@app.callback(
    Output("main-graph", "figure"),
    Output("console-output", "children"),
    Input("trigger-run-signal", "data"),
    Input("btn-run-custom-code", "n_clicks"),
    State("code-editor", "value"),
    State("console-output", "children"),
    prevent_initial_call=True
)
def execute_code(signal, n_clicks, code, current_console):
    ctx_id = ctx.triggered_id

    # Only run if triggered by signal (auto-run) or button click
    if not code:
        raise PreventUpdate

    try:
        # Create a safe local scope
        local_scope = {
            "pd": pd,
            "px": px,
            "go": go,
            "np": np,
            "figure_store": figure_store # Allow access to store
        }

        # Inject all dataframes into scope
        for name, df in figure_store.data_repository.items():
            local_scope[name] = df

        # Execute
        exec(code, {}, local_scope)

        # Check if 'fig' was created
        if "fig" in local_scope:
            fig = local_scope["fig"]
            if isinstance(fig, go.Figure):
                figure_store.update_figure(fig)
                return fig, f"{current_console}\n>>> Code executed successfully."
            else:
                return dash.no_update, f"{current_console}\n>>> Error: 'fig' is not a plotly Figure."
        else:
            return dash.no_update, f"{current_console}\n>>> Code executed, but no 'fig' variable found."

    except Exception as e:
        return dash.no_update, f"{current_console}\n>>> Execution Error: {e}"

---
## 5.4 Callback 实战分析：属性编辑器 (Property Editor)

---

### 🎯 核心设计目标

Property Editor 是整个应用最复杂的 Callback，实现了类似 Photoshop/Illustrator 的**属性面板编辑功能**：
- 选择画布上的任意元素（图表整体 / Trace / 注释）
- 动态显示该元素可编辑的属性
- 应用更改并实时更新画布

---

### 📐 架构设计图

```
┌─────────────────────────────────────────────────────────────────────────┐
│                   Property Editor 系统架构                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  用户选择元素:  dd-element-select                                        │
│       │         ["figure", "trace-0", "trace-1", "annot-0", ...]        │
│       ↓                                                                 │
│  ┌────────────────────────────────────────────────────────────────┐     │
│  │              populate_inspector_controls()                      │     │
│  │  根据元素类型动态生成属性控件:                                      │     │
│  │                                                                 │     │
│  │  if element == "figure":                                        │     │
│  │      → 生成: Title, Width, Height, Template, BgColor...        │     │
│  │                                                                 │     │
│  │  elif element.startswith("trace-"):                             │     │
│  │      → 生成: Name, Color, Size, Symbol, Mode...                 │     │
│  │      → 根据 trace_type 显示特定控件                              │     │
│  │                                                                 │     │
│  │  elif element.startswith("annot-"):                             │     │
│  │      → 生成: Text, X, Y, Font, Arrow...                         │     │
│  └────────────────────────────────────────────────────────────────┘     │
│       │                                                                 │
│       ↓                                                                 │
│  用户编辑 → 点击 "Apply" 按钮                                            │
│       │                                                                 │
│       ↓                                                                 │
│  ┌────────────────────────────────────────────────────────────────┐     │
│  │              apply_property_changes()                           │     │
│  │                                                                 │     │
│  │  1. 收集所有非空属性值 → props 字典                              │     │
│  │  2. 根据 selected_element 分发更新:                              │     │
│  │                                                                 │     │
│  │     "figure" → fig.update_layout(...)                          │     │
│  │     "trace-N" → fig.data[N].update(...)                        │     │
│  │     "annot-N" → fig.layout.annotations[N].update(...)          │     │
│  │                                                                 │     │
│  │  3. 保存到 figure_store 并返回                                   │     │
│  └────────────────────────────────────────────────────────────────┘     │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

### 🔑 关键技术解析

#### 1. 36 个 State 参数的处理

```python
@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),  # ← 必须！
    Input("btn-apply-props", "n_clicks"),   # 唯一触发器
    State("dd-element-select", "value"),    # 选中元素
    State("input-prop-name", "value"),      # 属性 1
    State("input-prop-color", "value"),     # 属性 2
    # ... 共 36 个 State ...
)
def apply_property_changes(n_clicks, selected_element, name, color, ...):
    # 收集非空值
    props = {}
    if name: props['name'] = name
    if color: props['color'] = color
    # ...
```

**设计考量**:
- 只用一个 Input (Apply 按钮)，其他全是 State
- 这样用户可以修改多个属性后一次性应用

#### 2. allow_duplicate=True 的用法

```python
Output("main-graph", "figure", allow_duplicate=True)
```

**为什么需要？**
- 多个 Callback 都输出到同一个组件 (`main-graph.figure`)
- 默认情况下 Dash 会报错（同一 Output 不能被多个 Callback 使用）
- `allow_duplicate=True` 解除这个限制

#### 3. 元素类型分发逻辑

```python
if selected_element == "figure":
    # 更新图表整体布局
    fig.update_layout(title=..., template=..., ...)
    
elif selected_element.startswith("trace-"):
    # 更新特定 Trace
    idx = int(selected_element.split("-")[1])
    trace = fig.data[idx]
    trace.update(marker=dict(color=..., size=...), ...)
    
elif selected_element.startswith("annot-"):
    # 更新特定注释
    idx = int(selected_element.split("-")[1])
    annotations = list(fig.layout.annotations)
    annotations[idx].update(text=..., font=..., ...)
    fig.update_layout(annotations=annotations)
```

---

### 🎨 不同元素的属性对照表

```
┌─────────────────┬──────────────────────────────────────────────────────┐
│  元素类型        │  可编辑属性                                          │
├─────────────────┼──────────────────────────────────────────────────────┤
│  figure         │  Title, Width, Height, Template, BgColor,            │
│  (图表整体)      │  ShowLegend, HoverMode, BarMode, AxisTitles          │
├─────────────────┼──────────────────────────────────────────────────────┤
│  trace-N        │  Name, Color, Size, Opacity, Symbol, LineWidth,       │
│  (数据系列)      │  LineDash, Mode, Fill, MarkerLineColor               │
├─────────────────┼──────────────────────────────────────────────────────┤
│  annot-N        │  Text, X, Y, Font, FontSize, FontColor, BgColor,     │
│  (注释)         │  ShowArrow, TextAngle                                 │
└─────────────────┴──────────────────────────────────────────────────────┘
```

---

### ❓ 可能的提问与回答

**Q1: 为什么用 36 个 State 而不是一个表单对象？**
> **A:** Dash 的设计哲学是"显式声明"。每个 State 对应一个 UI 控件，Dash 框架自动处理序列化/反序列化。用表单对象需要手动解析 JSON，更容易出错。

**Q2: allow_duplicate=True 有什么风险？**
> **A:** 竞态条件（Race Condition）！如果两个 Callback 同时更新同一个组件，结果不可预测。解决方法：
> - 确保这些 Callback 不会同时触发
> - 或者使用 `dcc.Store` 作为中间层统一管理状态

**Q3: clean_figure_dict() 函数是做什么的？**
> **A:** 清理从前端传回的 figure 字典！Plotly.js 有时会添加一些非标准字段（如 `selections`, `autosize` 等），这些在 Python 端重建 Figure 对象时可能导致错误。

**Q4: Trace 更新为什么用 `.update()` 而不是直接赋值？**
> **A:** 因为 Plotly 的 Trace 对象有复杂的嵌套结构！
> ```python
> # 错误做法 - 会覆盖其他 marker 属性
> trace.marker.color = 'red'
>
> # 正确做法 - 只更新指定属性
> trace.update(marker=dict(color='red'))
> ```

---

In [ ]:
# ====================================================================
# 12. Callbacks: Property Editor
# ====================================================================

@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Input("btn-apply-props", "n_clicks"),
    State("dd-element-select", "value"),
    State("input-prop-name", "value"), # Name/Title
    State("input-prop-color", "value"), # Color
    State("input-prop-size", "value"), # Size/Height
    State("input-prop-opacity", "value"), # Opacity
    State("input-prop-symbol", "value"), # Symbol
    State("input-prop-width", "value"), # Line Width/Width
    State("input-prop-dash", "value"), # Line Dash
    State("input-prop-mode", "value"), # Trace Mode
    State("input-prop-font", "value"), # Font Family
    State("input-prop-text", "value"), # Text
    State("input-prop-x", "value"), # X
    State("input-prop-y", "value"), # Y
    State("input-prop-template", "value"), # Figure Template
    State("input-prop-xaxis", "value"), # X Axis Title
    State("input-prop-yaxis", "value"), # Y Axis Title
    State("input-prop-legend", "value"), # Show Legend
    State("input-prop-fill", "value"), # Trace Fill
    State("input-prop-marker_line_color", "value"), # Marker Line Color (Fixed ID)
    State("input-prop-arrow", "value"), # Show Arrow
    State("input-prop-bgcolor", "value"), # Bg Color
    # --- NEW PROPERTIES (ROUND 3) ---
    State("input-prop-hovermode", "value"), # Hover Mode
    State("input-prop-grid_x", "value"), # Grid X (Fixed ID)
    State("input-prop-grid_y", "value"), # Grid Y (Fixed ID)
    State("input-prop-paper_color", "value"), # Paper Bg Color (Fixed ID)
    State("input-prop-line_shape", "value"), # Line Shape (Fixed ID)
    State("input-prop-text_pos", "value"), # Text Position (Fixed ID)
    State("input-prop-text_angle", "value"), # Text Angle (Fixed ID)
    # --- NEW PROPERTIES (ROUND 4) ---
    State("input-prop-legend_orient", "value"), # Legend Orientation (Fixed ID)
    State("input-prop-legend_pos", "value"), # Legend Position (Fixed ID)
    State("input-prop-barmode", "value"), # Bar Mode
    State("input-prop-log_x", "value"), # Log Scale X (Fixed ID)
    State("input-prop-log_y", "value"), # Log Scale Y (Fixed ID)
    State("input-prop-spikes", "value"), # Show Spikes
    State("input-prop-zeroline", "value"), # Show Zero Line
    State("input-prop-global_font_size", "value"), # Global Font Size (Fixed ID)

    State("main-graph", "figure"),
    prevent_initial_call=True
)
def apply_property_changes(n_clicks, selected_element, name, color, size, opacity, symbol, width, dash_style, mode, font, text, x, y,
                           template, xaxis_title, yaxis_title, show_legend, fill, marker_line_color, show_arrow, bgcolor,
                           hovermode, grid_x, grid_y, paper_color, line_shape, text_pos, text_angle,
                           legend_orient, legend_pos, barmode, log_x, log_y, spikes, zeroline, global_font_size,
                           fig_dict):
    if not n_clicks or not selected_element:
        raise PreventUpdate

    fig_dict = clean_figure_dict(fig_dict)
    fig = go.Figure(fig_dict)

    # Collect valid updates
    props = {}
    if name: props['name'] = name
    if color: props['color'] = color
    if size: props['size'] = size
    if opacity is not None: props['opacity'] = opacity
    if symbol: props['symbol'] = symbol
    if width: props['width'] = width
    if dash_style: props['dash'] = dash_style
    if mode: props['mode'] = mode
    if font: props['font'] = font
    if text: props['text'] = text
    if x is not None: props['x'] = x
    if y is not None: props['y'] = y
    if template: props['template'] = template
    if xaxis_title: props['xaxis_title'] = xaxis_title
    if yaxis_title: props['yaxis_title'] = yaxis_title
    if show_legend: props['showlegend'] = (show_legend == 'Show')
    if fill: props['fill'] = fill
    if marker_line_color: props['marker_line_color'] = marker_line_color
    if show_arrow: props['showarrow'] = (show_arrow == 'Show')
    if bgcolor: props['bgcolor'] = bgcolor
    # New props R3
    if hovermode: props['hovermode'] = hovermode
    if grid_x: props['grid_x'] = (grid_x == 'Show')
    if grid_y: props['grid_y'] = (grid_y == 'Show')
    if paper_color: props['paper_color'] = paper_color
    if line_shape: props['line_shape'] = line_shape
    if text_pos: props['text_pos'] = text_pos
    if text_angle is not None: props['text_angle'] = text_angle
    # New props R4
    if legend_orient: props['legend_orient'] = legend_orient
    if legend_pos: props['legend_pos'] = legend_pos
    if barmode: props['barmode'] = barmode
    if log_x: props['log_x'] = log_x
    if log_y: props['log_y'] = log_y
    if spikes: props['spikes'] = (spikes == 'Show')
    if zeroline: props['zeroline'] = (zeroline == 'Show')
    if global_font_size: props['global_font_size'] = global_font_size

    if not props: return dash.no_update

    if selected_element == "figure":
        layout_updates = {}
        if 'name' in props: layout_updates['title'] = dict(text=props['name'])
        if 'width' in props: layout_updates['width'] = int(props['width'])
        if 'size' in props: layout_updates['height'] = int(props['size']) # Map size to height for figure
        if 'color' in props: layout_updates['plot_bgcolor'] = props['color']
        if 'paper_color' in props: layout_updates['paper_bgcolor'] = props['paper_color']
        if 'font' in props:
            if 'font' not in layout_updates: layout_updates['font'] = {}
            layout_updates['font']['family'] = props['font']
        if 'global_font_size' in props:
            if 'font' not in layout_updates: layout_updates['font'] = {}
            layout_updates['font']['size'] = int(props['global_font_size'])

        if 'template' in props: layout_updates['template'] = props['template']
        if 'showlegend' in props: layout_updates['showlegend'] = props['showlegend']
        if 'hovermode' in props: layout_updates['hovermode'] = props['hovermode']
        if 'barmode' in props: layout_updates['barmode'] = props['barmode']

        # Legend Updates
        legend_updates = {}
        if 'legend_orient' in props: legend_updates['orientation'] = props['legend_orient']
        if 'legend_pos' in props:
            pos = props['legend_pos']
            if pos == 'tr': legend_updates.update(x=1.02, y=1, xanchor='left', yanchor='top')
            elif pos == 'tl': legend_updates.update(x=0, y=1, xanchor='left', yanchor='top')
            elif pos == 'br': legend_updates.update(x=1.02, y=0, xanchor='left', yanchor='bottom')
            elif pos == 'bl': legend_updates.update(x=0, y=0, xanchor='left', yanchor='bottom')
        if legend_updates: layout_updates['legend'] = legend_updates

        # Axis updates need careful merging
        xaxis_opts = {}
        if 'xaxis_title' in props: xaxis_opts['title'] = props['xaxis_title']
        if 'grid_x' in props: xaxis_opts['showgrid'] = props['grid_x']
        if 'log_x' in props: xaxis_opts['type'] = props['log_x']
        if 'spikes' in props: xaxis_opts['showspikes'] = props['spikes']
        if 'zeroline' in props: xaxis_opts['zeroline'] = props['zeroline']
        if xaxis_opts:
            fig.update_xaxes(**xaxis_opts)

        yaxis_opts = {}
        if 'yaxis_title' in props: yaxis_opts['title'] = props['yaxis_title']
        if 'grid_y' in props: yaxis_opts['showgrid'] = props['grid_y']
        if 'log_y' in props: yaxis_opts['type'] = props['log_y']
        if 'spikes' in props: yaxis_opts['showspikes'] = props['spikes']
        if 'zeroline' in props: yaxis_opts['zeroline'] = props['zeroline']
        if yaxis_opts:
            fig.update_yaxes(**yaxis_opts)

        fig.update_layout(**layout_updates)

    elif selected_element.startswith("trace_"):
        idx = int(selected_element.split("_")[1])
        if idx < len(fig.data):
            trace = fig.data[idx]
            updates = {}

            if 'name' in props: updates['name'] = props['name']
            if 'opacity' in props: updates['opacity'] = props['opacity']
            if 'mode' in props: updates['mode'] = props['mode']
            if 'fill' in props: updates['fill'] = props['fill'] if props['fill'] != 'none' else None
            if 'text_pos' in props: updates['textposition'] = props['text_pos']

            # Marker updates
            marker_updates = {}
            if 'color' in props: marker_updates['color'] = props['color']
            if 'size' in props: marker_updates['size'] = props['size']
            if 'symbol' in props: marker_updates['symbol'] = props['symbol']
            if 'width' in props: marker_updates['line'] = dict(width=props['width'])
            if 'marker_line_color' in props:
                if 'line' not in marker_updates: marker_updates['line'] = {}
                marker_updates['line']['color'] = props['marker_line_color']
            if marker_updates: updates['marker'] = marker_updates

            # Line updates
            line_updates = {}
            if 'color' in props: line_updates['color'] = props['color']
            if 'width' in props: line_updates['width'] = props['width']
            if 'dash' in props: line_updates['dash'] = props['dash']
            if 'line_shape' in props: line_updates['shape'] = props['line_shape']
            if line_updates: updates['line'] = line_updates

            trace.update(updates)

    elif selected_element.startswith("annot_"):
        idx = int(selected_element.split("_")[1])
        if fig.layout.annotations and idx < len(fig.layout.annotations):
            annot = fig.layout.annotations[idx]
            updates = {}
            if 'text' in props: updates['text'] = props['text']
            if 'text_angle' in props: updates['textangle'] = props['text_angle']

            font_updates = {}
            if 'color' in props: font_updates['color'] = props['color']
            if 'size' in props: font_updates['size'] = props['size']
            if 'font' in props: font_updates['family'] = props['font']
            if font_updates: updates['font'] = font_updates

            if 'x' in props: updates['x'] = props['x']
            if 'y' in props: updates['y'] = props['y']
            if 'showarrow' in props: updates['showarrow'] = props['showarrow']
            if 'bgcolor' in props: updates['bgcolor'] = props['bgcolor']

            annot.update(updates)

    elif selected_element.startswith("shape_"):
        idx = int(selected_element.split("_")[1])
        if fig.layout.shapes and idx < len(fig.layout.shapes):
            shape = fig.layout.shapes[idx]
            updates = {}
            line_updates = {}
            if 'color' in props: line_updates['color'] = props['color']
            if 'dash' in props: line_updates['dash'] = props['dash']
            if 'width' in props: line_updates['width'] = props['width']
            if line_updates: updates['line'] = line_updates

            if 'color' in props: updates['fillcolor'] = props['color']
            if 'opacity' in props: updates['opacity'] = props['opacity']
            if 'bgcolor' in props: updates['fillcolor'] = props['bgcolor']

            shape.update(updates)

    elif selected_element.startswith("image_"):
        idx = int(selected_element.split("_")[1])
        if fig.layout.images and idx < len(fig.layout.images):
            img = fig.layout.images[idx]
            updates = {}
            if 'opacity' in props: updates['opacity'] = props['opacity']
            if 'size' in props: updates['sizex'] = props['size']; updates['sizey'] = props['size']
            if 'x' in props: updates['x'] = props['x']
            if 'y' in props: updates['y'] = props['y']
            img.update(updates)

    figure_store.update_figure(fig)
    return fig

# New Callback: Highlight Selected Element (Auto-trigger on selection)
@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Input("btn-highlight", "n_clicks"),
    Input("dd-element-select", "value"), # Auto-highlight on selection
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def highlight_element(n_clicks, selected_element, fig_dict):
    if not selected_element or not fig_dict:
        raise PreventUpdate

    fig_dict = clean_figure_dict(fig_dict)
    fig = go.Figure(fig_dict)

    # Preserve UI state (zoom/pan)
    fig.update_layout(uirevision=True)

    # RELOAD FROM STORE TO GET CLEAN STATE
    clean_fig = figure_store.get_figure_dict()
    if clean_fig:
        fig = go.Figure(clean_fig)
        fig.update_layout(uirevision=True) # Keep zoom

    if selected_element == "figure":
        return fig # Just return clean figure if "Figure" is selected

    if selected_element.startswith("trace_"):
        idx = int(selected_element.split("_")[1])
        # Dim all other traces
        for i, trace in enumerate(fig.data):
            if i != idx:
                trace.update(opacity=0.2)
            else:
                trace.update(opacity=1.0)
                # Add a marker border to highlight
                # Check if marker exists or just try to update it
                trace.update(marker=dict(line=dict(width=2, color='red')))

    elif selected_element.startswith("shape_"):
        idx = int(selected_element.split("_")[1])
        # Highlight shape with thick red border
        if fig.layout.shapes and idx < len(fig.layout.shapes):
            shape = fig.layout.shapes[idx]
            shape.update(line=dict(width=4, color="red"), opacity=1.0)

    elif selected_element.startswith("annot_"):
        idx = int(selected_element.split("_")[1])
        if fig.layout.annotations and idx < len(fig.layout.annotations):
            annot = fig.layout.annotations[idx]
            annot.update(bordercolor="red", borderwidth=2, bgcolor="rgba(255, 255, 0, 0.3)")

    return fig

In [ ]:
@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Output("dd-element-select", "value"), # Reset selection after delete
    Input("btn-delete-element", "n_clicks"),
    State("dd-element-select", "value"),
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def delete_element(n_clicks, selected_element, fig_dict):
    if not n_clicks or not selected_element or selected_element == "figure":
        raise PreventUpdate

    # Identify what to delete
    parts = selected_element.split("_")
    if len(parts) < 2:
        raise PreventUpdate

    el_type = parts[0]
    try:
        idx = int(parts[1])
    except ValueError:
        raise PreventUpdate

    # Perform deletion in store
    if el_type == "trace":
        figure_store.remove_trace(idx)
    elif el_type == "annot":
        figure_store.remove_annotation(idx)
    elif el_type == "shape":
        figure_store.remove_shape(idx)
    elif el_type == "image":
        figure_store.remove_image(idx)

    # Get updated figure
    new_fig = figure_store.get_figure_dict()
    if not new_fig:
        return dash.no_update, "figure"

    # Force a layout update to ensure removal is visualised immediately
    fig = go.Figure(new_fig)
    fig.update_layout(uirevision=datetime.now().isoformat())

    return fig, "figure"

---
## 5.5 Callback 实战分析：绘图工具 & 注释系统

---

### 🎯 功能概述

这一组 Callback 实现了类似 Adobe Illustrator 的**绘图模式切换**和**注释添加**功能：

| Callback 名称 | 触发器 | 功能 |
|--------------|--------|------|
| `set_shape_draw_mode()` | 5个绘图按钮 | 切换画布的 dragmode |
| `sync_drawn_shapes()` | main-graph.relayoutData | 同步绘制的形状到 store |
| `toggle_annot_modal()` | 添加文本按钮 | 打开/关闭注释弹窗 |
| `add_text_annotation()` | 确认按钮 | 创建新注释 |

---

### 📐 绘图模式切换机制

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     Plotly 的 dragmode 绘图模式                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│   用户点击绘图按钮:                                                       │
│                                                                         │
│   🔲 Line     → dragmode = "drawline"      → 画直线                      │
│   ⬜ Rect     → dragmode = "drawrect"      → 画矩形                      │
│   ⭕ Circle   → dragmode = "drawcircle"    → 画圆/椭圆                   │
│   〰️ Free     → dragmode = "drawopenpath"  → 自由曲线                    │
│   📐 Polygon  → dragmode = "drawclosedpath"→ 闭合多边形                  │
│                                                                         │
│   鼠标在画布上拖拽:                                                       │
│       ↓                                                                 │
│   Plotly.js 自动创建 Shape 对象                                          │
│       ↓                                                                 │
│   触发 relayoutData → sync_drawn_shapes() 保存到 FigureStore            │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

### 🔑 关键技术解析

#### 1. relayoutData - 画布交互事件监听

```python
@app.callback(
    Output("figure-store-client", "data", allow_duplicate=True),
    Output("main-graph", "figure", allow_duplicate=True),
    Input("main-graph", "relayoutData"),  # ← Plotly 的布局变化事件
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def sync_drawn_shapes(relayout_data, fig_dict):
    if 'shapes' in relayout_data:
        # 用户刚画完一个形状！
        fig.layout.shapes = relayout_data['shapes']
        figure_store.update_figure(fig)
```

**relayoutData 事件包含:**
- `shapes` - 形状列表更新
- `xaxis.range` / `yaxis.range` - 缩放事件
- `dragmode` - 拖拽模式变化

#### 2. Modal 弹窗模式

```python
@app.callback(
    Output("modal-annotation", "is_open"),
    Input("btn-add-text", "n_clicks"),
    Input("btn-confirm-annot", "n_clicks"),
    State("modal-annotation", "is_open"),
)
def toggle_annot_modal(n1, n2, is_open):
    # 任一按钮点击都会切换状态
    if n1 or n2: return not is_open
    return is_open
```

**Bootstrap Modal 组件:**
- `is_open=True` → 显示弹窗
- `is_open=False` → 隐藏弹窗
- 用一个 Callback 控制开关

#### 3. 注释定位 - paper 坐标系

```python
fig.add_annotation(
    text=text,
    x=0.5, y=0.5,      # 0-1 范围
    xref="paper",      # paper = 画布百分比坐标
    yref="paper",
    # ...
)
```

**Plotly 坐标系统:**
- `xref="paper"` → x=0 是左边缘, x=1 是右边缘
- `xref="x"` → x 使用数据坐标系

---

### ❓ 可能的提问与回答

**Q1: 绘制的形状为什么需要手动同步？**
> **A:** Plotly.js 在前端绘制形状，但 Python 后端的 FigureStore 不知道！`relayoutData` 事件是双向同步的桥梁。

**Q2: 为什么 Confirm 按钮既关闭弹窗又添加注释？**
> **A:** 用户体验！一键完成两个操作。实际上是两个 Callback 同时触发：
> - `toggle_annot_modal()` → 关闭弹窗
> - `add_text_annotation()` → 添加注释

---

In [ ]:
# ====================================================================
# 13. Callbacks: Drawing Tools & Annotations
# ====================================================================

@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Input("btn-draw-line", "n_clicks"),
    Input("btn-draw-rect", "n_clicks"),
    Input("btn-draw-circle", "n_clicks"),
    Input("btn-draw-free", "n_clicks"),
    Input("btn-draw-poly", "n_clicks"),
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def set_shape_draw_mode(n_line, n_rect, n_circle, n_free, n_poly, fig_dict):
    if not fig_dict: raise PreventUpdate
    ctx_id = ctx.triggered_id

    fig_dict = clean_figure_dict(fig_dict)
    fig = go.Figure(fig_dict)

    # Set dragmode to enable drawing
    if ctx_id == "btn-draw-line":
        fig.update_layout(dragmode="drawline")
    elif ctx_id == "btn-draw-rect":
        fig.update_layout(dragmode="drawrect")
    elif ctx_id == "btn-draw-circle":
        fig.update_layout(dragmode="drawcircle")
    elif ctx_id == "btn-draw-free":
        fig.update_layout(dragmode="drawopenpath")
    elif ctx_id == "btn-draw-poly":
        fig.update_layout(dragmode="drawclosedpath")

    # Ensure newshape properties are set for visibility
    fig.update_layout(newshape=dict(line=dict(color="black", width=2), opacity=1))

    figure_store.update_figure(fig)
    return fig

# Sync drawn shapes from graph to store AND update graph to reflect changes immediately
@app.callback(
    Output("figure-store-client", "data", allow_duplicate=True),
    Output("main-graph", "figure", allow_duplicate=True),
    Input("main-graph", "relayoutData"),
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def sync_drawn_shapes(relayout_data, fig_dict):
    if not relayout_data or not fig_dict:
        raise PreventUpdate

    # Check if shapes were added/modified
    if 'shapes' in relayout_data:
        # This happens when a shape is drawn or modified
        fig_dict = clean_figure_dict(fig_dict)
        fig = go.Figure(fig_dict)
        fig.layout.shapes = relayout_data['shapes']

        # Update store
        figure_store.update_figure(fig)

        # Return updated figure to graph to ensure consistency and trigger other callbacks
        return fig.to_dict(), fig

    return dash.no_update, dash.no_update

@app.callback(
    Output("modal-annotation", "is_open"),
    Input("btn-add-text", "n_clicks"),
    Input("btn-confirm-annot", "n_clicks"),
    State("modal-annotation", "is_open"),
    prevent_initial_call=True
)
def toggle_annot_modal(n1, n2, is_open):
    if n1 or n2: return not is_open
    return is_open

@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Input("btn-confirm-annot", "n_clicks"),
    State("annot-text", "value"),
    State("annot-x", "value"),
    State("annot-y", "value"),
    State("annot-arrow", "value"),
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def add_text_annotation(n_clicks, text, x, y, show_arrow, fig_dict):
    if not n_clicks or not text: raise PreventUpdate

    fig_dict = clean_figure_dict(fig_dict)
    fig = go.Figure(fig_dict)

    # Push to history
    history_stack.push(clean_figure_dict(fig.to_dict()))

    # Default to center if no coords
    xref, yref = "paper", "paper"
    x_val, y_val = (x, y) if (x is not None and y is not None) else (0.5, 0.5)

    fig.add_annotation(
        text=text,
        x=x_val, y=y_val,
        xref=xref, yref=yref,
        showarrow=bool(show_arrow),
        arrowhead=2 if show_arrow else 0,
        font=dict(size=14, color="black"),
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1
    )

    figure_store.update_figure(fig)
    return fig

In [ ]:
# ====================================================================
# 13b. Callbacks: Media Tools (Image Upload)
# ====================================================================

@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Input("upload-image", "contents"),
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def add_background_image(content, fig_dict):
    if not content:
        raise PreventUpdate

    fig_dict = clean_figure_dict(fig_dict)
    fig = go.Figure(fig_dict)

    # Add image to layout
    # content is already "data:image/png;base64,..."

    new_image = dict(
        source=content,
        xref="paper", yref="paper",
        x=0, y=1,
        sizex=1, sizey=1,
        sizing="stretch",
        opacity=0.5,
        layer="below"
    )

    fig.add_layout_image(new_image)

    figure_store.update_figure(fig)
    return fig

In [ ]:
# ====================================================================
# 14. Callbacks: History & Session
# ====================================================================

@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Output("btn-undo", "disabled"),
    Output("btn-redo", "disabled"),
    Input("btn-undo", "n_clicks"),
    Input("btn-redo", "n_clicks"),
    Input("main-graph", "figure"), # Listen to graph updates to update button state
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def manage_history(n_undo, n_redo, fig_trigger, current_fig_dict):
    ctx_id = ctx.triggered_id

    # If triggered by graph update, just update buttons
    if ctx_id == "main-graph":
        return dash.no_update, not history_stack.can_undo(), not history_stack.can_redo()

    # Handle Undo/Redo
    new_fig = None
    if ctx_id == "btn-undo":
        new_fig = history_stack.undo()
    elif ctx_id == "btn-redo":
        new_fig = history_stack.redo()

    if new_fig:
        new_fig = clean_figure_dict(new_fig)
        fig = go.Figure(new_fig)
        figure_store.update_figure(fig)
        return fig, not history_stack.can_undo(), not history_stack.can_redo()

    return dash.no_update, not history_stack.can_undo(), not history_stack.can_redo()

@app.callback(
    Output("download-component", "data"),
    Input("btn-save-session", "n_clicks"),
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def save_session(n_clicks, fig_dict):
    if not fig_dict: raise PreventUpdate
    fig_dict = clean_figure_dict(fig_dict)
    return dict(content=json.dumps(fig_dict, indent=2), filename="session.json")

@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Input("upload-session", "contents"),
    prevent_initial_call=True
)
def load_session(content):
    if not content: raise PreventUpdate
    try:
        content_type, content_string = content.split(',')
        decoded = base64.b64decode(content_string)
        fig_dict = json.loads(decoded.decode('utf-8'))
        fig_dict = clean_figure_dict(fig_dict)
        fig = go.Figure(fig_dict)
        figure_store.update_figure(fig)
        history_stack.push(fig_dict)
        return fig
    except Exception as e:
        print(f"Error loading session: {e}")
        raise PreventUpdate

---
## 5.6 Callback 实战分析：历史记录 & 会话管理

---

### 🎯 功能概述

这是实现"无限撤销/重做"的核心 Callback，同时支持会话导出/导入：

| Callback 名称 | 功能 | 输出 |
|--------------|------|------|
| `manage_history()` | Undo/Redo 操作 | 恢复历史图表 + 按钮状态 |
| `save_session()` | 导出当前会话 | 触发文件下载 |
| `load_session()` | 导入会话文件 | 恢复图表状态 |

---

### 📐 历史管理架构

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     manage_history() Callback                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  触发器 (Inputs):                                                        │
│  ├── btn-undo.n_clicks      → Undo 按钮点击                              │
│  ├── btn-redo.n_clicks      → Redo 按钮点击                              │
│  └── main-graph.figure      → 图表更新（用于同步按钮状态）                 │
│                                                                         │
│  输出 (Outputs):                                                         │
│  ├── main-graph.figure      → 恢复的历史图表                              │
│  ├── btn-undo.disabled      → Undo 按钮禁用状态                           │
│  └── btn-redo.disabled      → Redo 按钮禁用状态                           │
│                                                                         │
│  ┌────────────────────────────────────────────────────────────────┐     │
│  │                      处理逻辑                                   │     │
│  │                                                                 │     │
│  │  if ctx_id == "main-graph":                                     │     │
│  │      # 图表更新时只更新按钮状态，不执行 Undo/Redo                  │     │
│  │      return no_update, can_undo, can_redo                       │     │
│  │                                                                 │     │
│  │  if ctx_id == "btn-undo":                                       │     │
│  │      new_fig = history_stack.undo()                             │     │
│  │      figure_store.update_figure(new_fig)                        │     │
│  │                                                                 │     │
│  │  if ctx_id == "btn-redo":                                       │     │
│  │      new_fig = history_stack.redo()                             │     │
│  │      figure_store.update_figure(new_fig)                        │     │
│  └────────────────────────────────────────────────────────────────┘     │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

### 🔑 关键技术解析

#### 1. 多 Input 单 Callback 的"门户"模式

```python
@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Output("btn-undo", "disabled"),
    Output("btn-redo", "disabled"),
    Input("btn-undo", "n_clicks"),      # 触发器 1
    Input("btn-redo", "n_clicks"),      # 触发器 2
    Input("main-graph", "figure"),      # 触发器 3 (同步用)
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def manage_history(n_undo, n_redo, fig_trigger, current_fig_dict):
    ctx_id = ctx.triggered_id
    
    # 门户分发
    if ctx_id == "main-graph":
        # 只更新按钮状态
        return dash.no_update, not history_stack.can_undo(), not history_stack.can_redo()
    elif ctx_id == "btn-undo":
        # 执行撤销
        ...
```

#### 2. dcc.Download 组件实现文件下载

```python
@app.callback(
    Output("download-component", "data"),  # ← dcc.Download 组件
    Input("btn-save-session", "n_clicks"),
    State("main-graph", "figure"),
)
def save_session(n_clicks, fig_dict):
    return dict(
        content=json.dumps(fig_dict, indent=2),  # JSON 内容
        filename="session.json"                   # 下载文件名
    )
```

**dcc.Download 工作原理:**
- 当 `data` 属性被设置时，浏览器自动下载文件
- 支持 `content` (字符串) 或 `base64` (二进制)

#### 3. 会话恢复的完整流程

```python
def load_session(content):
    # 1. 解码 Base64
    content_type, content_string = content.split(',')
    decoded = base64.b64decode(content_string)
    
    # 2. 解析 JSON
    fig_dict = json.loads(decoded.decode('utf-8'))
    
    # 3. 清理非标准字段
    fig_dict = clean_figure_dict(fig_dict)
    
    # 4. 重建 Figure
    fig = go.Figure(fig_dict)
    
    # 5. 同步到 Store
    figure_store.update_figure(fig)
    
    # 6. 添加到历史栈
    history_stack.push(fig_dict)
    
    return fig
```

---

### ❓ 可能的提问与回答

**Q1: 为什么 manage_history 要监听 main-graph.figure？**
> **A:** 用于**按钮状态同步**！当图表被其他 Callback 更新时（如添加 Trace），需要重新计算 can_undo/can_redo 状态来更新按钮。

**Q2: 会话文件包含哪些信息？**
> **A:** 完整的 Plotly Figure 字典，包括：
> - `data` - 所有 Trace 的配置
> - `layout` - 布局、坐标轴、注释、形状等
> - 用户可以用这个 JSON 在任何支持 Plotly 的环境重建图表！

**Q3: 为什么要同时更新 figure_store 和 history_stack？**
> **A:** 职责分离！
> - `figure_store` - 当前状态（供其他 Callback 读取）
> - `history_stack` - 历史记录（供 Undo/Redo 使用）

---

In [ ]:
# ====================================================================
# 15. Callbacks: View Tools
# ====================================================================

@app.callback(
    Output("main-graph", "figure", allow_duplicate=True),
    Input("btn-tool-zoom", "n_clicks"),
    Input("btn-tool-pan", "n_clicks"),
    Input("btn-tool-reset", "n_clicks"),
    State("main-graph", "figure"),
    prevent_initial_call=True
)
def view_tools(n_zoom, n_pan, n_reset, fig_dict):
    ctx_id = ctx.triggered_id
    if not fig_dict: raise PreventUpdate

    fig_dict = clean_figure_dict(fig_dict)
    fig = go.Figure(fig_dict)

    if ctx_id == "btn-tool-zoom":
        fig.update_layout(dragmode="zoom")
    elif ctx_id == "btn-tool-pan":
        fig.update_layout(dragmode="pan")
    elif ctx_id == "btn-tool-reset":
        fig.update_xaxes(autorange=True)
        fig.update_yaxes(autorange=True)
        if 'scene' in fig.layout:
            fig.update_scenes(xaxis_autorange=True, yaxis_autorange=True, zaxis_autorange=True)

    figure_store.update_figure(fig)
    return fig

In [ ]:
# ====================================================================
# 15b. Callbacks: Real-time Statistics
# ====================================================================

@app.callback(
    Output("console-output", "children", allow_duplicate=True),
    Input("main-graph", "selectedData"),
    State("console-output", "children"),
    prevent_initial_call=True
)
def show_selection_stats(selected_data, current_console):
    if not selected_data:
        raise PreventUpdate

    points = selected_data.get("points", [])
    if not points:
        return dash.no_update

    msg = f">>> Selected {len(points)} points."

    # Calculate simple stats from the selection data directly
    ys = [p.get('y') for p in points if 'y' in p and isinstance(p.get('y'), (int, float))]
    if ys:
        mean_y = sum(ys) / len(ys)
        min_y = min(ys)
        max_y = max(ys)
        msg += f" | Y-Stats: Mean={mean_y:.2f}, Min={min_y:.2f}, Max={max_y:.2f}"

    # Keep console history reasonable
    if len(current_console) > 1000:
        current_console = current_console[-500:]

    return f"{current_console}\n{msg}"

In [ ]:
# ====================================================================
# 16. Launch Application
# ====================================================================

if __name__ == '__main__':
    print("\n" + "="*72)
    print("🚀 Python Interactive Figure Editor - Starting...")
    print("="*72)
    print("📍 URL: http://localhost:8055")
    print("💡 Tip: Use Ctrl+Click to open in browser")
    print("⚡ Feature Highlights:")
    print("   - Dash-powered canvas with MATLAB-style figure editing")
    print("   - Drawing tools (line/rect/circle/freehand) + undo/redo stack")
    print("   - Trace styling, theme presets, and live property inspector")
    print("   - Lasso statistics & outlier removal from datasets")
    print("   - Hybrid canvas: overlay images with adjustable opacity")
    print("   - Layer manager with visibility toggles and summaries")
    print("   - Code generator + session export/restore + PNG output")
    print("="*72 + "\n")
    app.run(debug=True, jupyter_mode='inline', port=8055)


🚀 Python Interactive Figure Editor - Starting...
📍 URL: http://localhost:8055
💡 Tip: Use Ctrl+Click to open in browser
⚡ Feature Highlights:
   - Dash-powered canvas with MATLAB-style figure editing
   - Drawing tools (line/rect/circle/freehand) + undo/redo stack
   - Trace styling, theme presets, and live property inspector
   - Lasso statistics & outlier removal from datasets
   - Hybrid canvas: overlay images with adjustable opacity
   - Layer manager with visibility toggles and summaries
   - Code generator + session export/restore + PNG output



---

# Part 6: 应用启动与部署

---

## 6.1 Dash 应用启动机制

### 🎯 `app.run()` 参数详解

| 参数 | 值 | 说明 |
|------|-----|------|
| `debug=True` | 开发模式 | 启用热重载 + 详细错误信息 |
| `jupyter_mode='inline'` | Jupyter 内嵌 | 在 Notebook 单元格内显示 |
| `port=8051` | 端口号 | 避免与其他服务冲突 |

---

### 📐 Dash 应用生命周期

```
┌─────────────────────────────────────────────────────────────────────────┐
│                      Dash 应用启动流程                                   │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  1. Python 初始化阶段                                                    │
│     ├── 创建 Dash 实例 → app = Dash(__name__)                           │
│     ├── 初始化全局单例 → figure_store, history_stack, code_generator    │
│     ├── 构建 UI 组件 → ribbon, workspace_panel, property_inspector      │
│     └── 设置 app.layout → HTML 结构                                     │
│                                                                         │
│  2. 启动 Flask 服务器                                                    │
│     ├── app.run() → 启动 Flask                                          │
│     ├── 监听 http://localhost:8051                                      │
│     └── 等待客户端请求                                                   │
│                                                                         │
│  3. 浏览器请求阶段                                                       │
│     ├── GET / → 返回 HTML 页面                                          │
│     ├── 加载 React.js + Plotly.js                                       │
│     └── 渲染 UI 组件                                                     │
│                                                                         │
│  4. Callback 执行阶段 (循环)                                             │
│     ├── 用户交互触发 Input                                               │
│     ├── AJAX POST /_dash-update-component                               │
│     ├── Flask 路由 → 执行 Python Callback                               │
│     ├── 返回 JSON → React 更新 DOM                                       │
│     └── 重复...                                                         │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

### 🔧 Debug 模式特性

```python
app.run(debug=True, ...)
```

**开启的功能:**
1. **热重载 (Hot Reload)** - 修改代码后自动刷新
2. **回调调试器** - 显示 Callback 执行图
3. **详细错误追踪** - 完整的 Python Traceback
4. **性能分析** - 每个 Callback 的执行时间

**⚠️ 生产环境警告:**
```python
# 生产部署时必须关闭！
app.run(debug=False, host='0.0.0.0', port=80)
```

---



### ❓ 可能的提问与回答

**Q1: 为什么用 port=8051 而不是默认的 8050？**
> **A:** 避免端口冲突！如果同时运行多个 Dash 应用，需要使用不同端口。

**Q2: jupyter_mode='inline' 和 'external' 有什么区别？**
> **A:**
> - `inline` - 在 Notebook 单元格内嵌显示（教学演示常用）
> - `external` - 在新浏览器标签页打开（更大的操作空间）
> - `tab` - 在 Jupyter 的新标签页打开

**Q3: 为什么 if __name__ == '__main__' 是必要的？**
> **A:** Python 惯例！确保这段代码只在"直接运行文件时"执行，而不是在被 `import` 时执行。

**Q4: 如何在生产环境部署这个应用？**
> **A:** 推荐使用 Gunicorn + Nginx:
> ```bash
> gunicorn app:server --workers 4 --bind 0.0.0.0:8000
> ```
> 其中 `server = app.server` 暴露底层 Flask 实例。

---

## 🎉 项目总结

### 技术栈回顾

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    PyFigureEditor 技术架构                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  前端层 (JavaScript)                                                     │
│  ├── React.js ← Dash 自动管理                                           │
│  ├── Plotly.js ← 图表渲染引擎                                            │
│  └── Bootstrap 5 ← UI 组件库                                             │
│                                                                         │
│  后端层 (Python)                                                         │
│  ├── Flask ← Web 服务器                                                  │
│  ├── Dash ← 响应式框架                                                   │
│  └── Pandas/NumPy ← 数据处理                                             │
│                                                                         │
│  设计模式                                                                │
│  ├── Singleton ← 全局状态管理 (figure_store, history_stack)              │
│  ├── Repository ← 数据仓储 (data_repository)                             │
│  ├── DTO ← 数据传输对象 (TraceDataset)                                   │
│  └── Memento ← 历史快照 (HistoryStack)                                   │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### 核心 Callback 数量统计

| 类别 | 数量 | 示例 |
|------|------|------|
| 数据管理 | 3 | manage_data, workspace_tabs |
| 代码生成 | 2 | generate_plot, execute_code |
| 属性编辑 | 3 | apply_props, populate_controls, highlight |
| 绘图工具 | 4 | draw_mode, sync_shapes, annotation |
| 历史管理 | 3 | undo/redo, save/load session |
| 视图工具 | 2 | zoom/pan/reset, selection_stats |
| **总计** | **~17** | |

---
